In [129]:
from CartPole import CartPole, remap_angle, cartpole_step, remap_angle2
import numpy as np
import matplotlib
import pandas as pd
import seaborn as sns
matplotlib.use('TkAgg') 
import scipy

import matplotlib.pyplot as plt
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)

import pickle

In [2]:
# important helper functions for all tasks
def get_std(X):
    return np.std(X, axis=0)

def convert_dict_to_array(data):
    # zip all the values in the dictionary together and convert it to a numpy array
    # suppose the keys are not known beforehand
    keys = list(data.keys())
    values = [data[key] for key in keys]
    return np.array(list(zip(*values)))



---
## Task 1.1

We are assuming the system is a markov state. This can be shown in this case from the equations of motion. 

We are also assuming that the system is time invariant. The system dynamics can be started at any time instant and will still behave the right way

Maybe use a heatmap to show the initial conditions that lead to flip over

In [96]:
def rollout(initial_state, initial_force, num_steps, visual=True, max_force=20):
    """
    Simulate the CartPole environment for a given number of steps.
    
    Args:
        initial_state (tuple): The initial state of the environment.
        it should be a tuple of the form (cart_location, cart_velocity, 
                                        pole_angle, pole_velocity).

        initial_force (float): The initial force applied to the cart.
        num_steps (int): The number of steps to simulate.
    
    Returns:
        data: A dictionary containing the cart location, cart velocity, 
              pole angle and pole angular velocity at each step.
    """
    env = CartPole(visual=visual, max_force=max_force)
    env.reset()

    data = {'cart_location': [],
            'cart_velocity': [],
            'pole_angle': [],
            'pole_velocity': []
        }
    
    # Set the initial state
    env.setState(initial_state)

    # Perform the action for the specified number of steps
    for step in range(num_steps + 1):
        # Store the current state
        data['cart_location'].append(env.cart_location)
        data['cart_velocity'].append(env.cart_velocity)
        data['pole_angle'].append(env.pole_angle)
        data['pole_velocity'].append(env.pole_velocity)

        # Perform the action
        env.performAction(initial_force)

        # remap the angle to be between -pi and pi
        env.remap_angle()
    
    # close the plot
    if visual:
        env.close_plot()
        plt.close()
        
    return data

    

In [ ]:
# Plot functions

# plot y vs x
def plot_x_y(x, y, xlabel, ylabel):
    plt.figure()
    ax = plt.gca()
    plt.xlim(min(x) - 0.1, max(x) + 0.1)
    plt.ylim(min(y) - 0.1, max(y) + 0.1)
    ax.plot(x, y, 'r-')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(f'{ylabel} vs {xlabel}')
    plt.grid()
    plt.show()

# plot y vs time
def plot_y(y, ylabel):
    plt.figure()
    ax = plt.gca()
    plt.xlim(0, len(y) + 2)
    plt.ylim(min(y) - 0.1, max(y) + 0.1)
    ax.plot(np.arange(0, len(y)),y, 'r-')
    plt.ylabel(ylabel)
    plt.title(f'{ylabel} vs Iterations')
    plt.xlabel('Iterations')
    plt.grid()
    plt.show()


# plot multiple y vs time
def plot_data_time(data, verdict):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 5))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            ax[i, j].plot(np.arange(0, len(data[title])), data[title], 'r-')
            ax[i, j].set_title(title + ' vs Iterations')
            ax[i, j].set_xlabel('Iterations')
            ax[i, j].set_ylabel(title)
            ax[i, j].grid()
    # center the title on top of the figure
    fig.suptitle(f"Initial Carriage velocity: {data['cart_velocity'][0]:.2f} m/s \nPole angular velocity: {data['pole_velocity'][0]:.2f} rad/s \n{verdict}")
    fig.tight_layout(rect=[0, 0.03, 1, 1])  # Adjust the rect to make space for the title
    plt.show()

# plot multiple y vs x
def plot_state(x_data, y_data):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 5))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            ax[i, j].plot(x_data[title], y_data[title], 'r-')
            ax[i, j].set_title("change in " + title + ' vs ' + title)
            ax[i, j].set_xlabel(title)
            ax[i, j].set_ylabel(title)
            ax[i, j].set_xlim(min(-1, min(x_data[title]) - 0.1), max(1, max(x_data[title]) + 0.1))
            ax[i, j].set_ylim(min(-1, min(y_data[title]) - 0.1), max(1, max(y_data[title]) + 0.1))
            ax[i, j].grid()
    # center the title on top of the figure
    fig.suptitle("Change in state vs State")
    fig.tight_layout(rect=[0, 0.05, 0.95, 1])  # Adjust the rect to make space for the title
    plt.show()

# plot multiple y vs x
def plot_pair_plot(data, verdict):
    fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(15, 12))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(3):
        title_x = titles[i]
        for j in range(2):
            if i + j + 1 >= len(titles):
                title_y = titles[0]
                title_x = titles[-1]
            else:
                title_y = titles[i + j + 1]
            ax[i, j].plot(data[title_x], data[title_y], 'r-')
            ax[i, j].set_title(title_y + ' vs ' + title_x)
            ax[i, j].set_xlabel(title_x)
            ax[i, j].set_ylabel(title_y)
            ax[i, j].grid()
    # center the title on top of the figure
    fig.suptitle(f"Initial Carriage velocity: {data['cart_velocity'][0]:.2f} m/s \nPole angular velocity: {data['pole_velocity'][0]:.2f} rad/s \n{verdict}")
    fig.tight_layout(rect=[0, 0.05, 0.95, 1])  # Adjust the rect to make space for the title
    plt.show()

# plot multiple y vs x
def plot_phase_plot(data, verdict):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for j in range(2):
        title_x = titles[j * 2]
        title_y = titles[j * 2 + 1]
        ax[j].plot(data[title_x], data[title_y], 'r-')
        ax[j].set_title(title_y + ' vs ' + title_x)
        ax[j].set_xlabel(title_x)
        ax[j].set_ylabel(title_y)
        ax[j].grid()
    # center the title on top of the figure
    fig.suptitle(f"Initial Carriage velocity: {data['cart_velocity'][0]:.2f} m/s \nPole angular velocity: {data['pole_velocity'][0]:.2f} rad/s \n{verdict}")
    fig.tight_layout(rect=[0, 0.05, 0.95, 1])  # Adjust the rect to make space for the title
    plt.show()

In [ ]:
# different experiments
# Experiment 1: Rollout with initial state (0, 0, pi, 5) and initial force 0
data = rollout(initial_state=(0, 0, np.pi, 5), initial_force=0, num_steps=100)
plot_data_time(data, "Simple osscillation of pole")
plot_phase_plot(data, "Simple osscillation of pole")

In [ ]:
# Experiment 2: Rollout with initial state (0, 0, pi, 14) and initial force 0
data = rollout(initial_state=(0, 0, np.pi, 13.85), initial_force=0, num_steps=100)
plot_data_time(data, "Complete rotation")
plot_phase_plot(data, "Complete rotation")

In [ ]:
# Experiment 3: Rollout with initial state (0, 1, pi, 8) and initial force 0
data = rollout(initial_state=(0, 1, np.pi, 8), initial_force=0, num_steps=100)
plot_data_time(data, "Oscillation of pole and cart")
plot_phase_plot(data, "Oscillation of pole and cart")

In [ ]:
# Experiment 4: Rollout with initial state (0, 10, pi, 14) and initial force 0
data = rollout(initial_state=(0, 10, np.pi, 14), initial_force=0, num_steps=100)
plot_data_time(data, "complete rotation")
plot_phase_plot(data, "complete rotation")

In [ ]:

def count_revolution(initial_state, initial_force, num_steps):
    """
    Simulate the CartPole environment for a given number of steps.
    
    Args:
        initial_state (tuple): The initial state of the environment.
        it should be a tuple of the form (cart_location, cart_velocity, 
                                        pole_angle, pole_velocity).

        initial_force (float): The initial force applied to the cart.
        num_steps (int): The number of steps to simulate.
    
    Returns:
        revolution: The number of revolutions made by the pole.
    """
    env = CartPole(visual=False)
    env.reset()
    
    # Set the initial state
    env.setState(initial_state)

    num_revolutions = 0
    new_revolution = True

    # Perform the action for the specified number of steps
    for step in range(num_steps + 1):

        # Perform the action
        env.performAction(initial_force)

        # remap the angle to be between -pi and pi
        if abs(env.pole_angle) < 0.3:
            if new_revolution:
                # count the number of revolutions
                num_revolutions += 1

            new_revolution = False
        else:
            new_revolution = True

        env.remap_angle()
    
    if num_revolutions > 1:
        num_revolutions -= 1

    return num_revolutions

def generate_heatmap(num_steps, initial_force = 0):
    cart_velocity = np.linspace(-10, 10, num_steps)
    pole_velocity= np.linspace(-15, 15, num_steps)

    initial_state = [0, 0, np.pi, 0]

    revolution = [[0 for i in range(num_steps)] for j in range(num_steps)]

    for i, cv in enumerate(cart_velocity):
        for j, pv in enumerate(pole_velocity):
            initial_state[1] = cv
            initial_state[3] = pv

            num_revolutions = count_revolution(initial_state, initial_force, num_steps)

            revolution[i][j] = num_revolutions
    
    revolution = np.array(revolution)
    df = pd.DataFrame(revolution, index = cart_velocity, columns=pole_velocity)
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.heatmap(df, annot=False, fmt=".1f", cmap="coolwarm")

    # Set axis labels
    ax.set_xlabel("initial pole velocity", fontsize=14)
    ax.set_ylabel("initial cart velocity", fontsize=14)
    ax.set_title("Revolutions", fontsize=16)

    # Set ticks at regular intervals (e.g., every 5 steps)
    step = 5  # Show every 5th tick label

    xticks = np.arange(0, num_steps, step)
    yticks = np.arange(0, num_steps, step)

    ax.set_xticks(xticks + 0.5)
    ax.set_xticklabels([f"{pole_velocity[i]:.2f}" for i in xticks], rotation=45, fontsize=12)

    ax.set_yticks(yticks + 0.5)
    ax.set_yticklabels([f"{cart_velocity[i]:.2f}" for i in yticks], rotation=0, fontsize=12)


    # Show the plot
    plt.show()


generate_heatmap(num_steps=50)          

    

In [ ]:
def generate_heatmap_zoomed(num_steps, initial_force = 0):
    cart_velocity = np.linspace(8, 10, num_steps)
    pole_velocity= np.linspace(13, 15, num_steps)

    initial_state = [0, 0, np.pi, 0]

    revolution = [[0 for i in range(num_steps)] for j in range(num_steps)]

    for i, cv in enumerate(cart_velocity):
        for j, pv in enumerate(pole_velocity):
            initial_state[1] = cv
            initial_state[3] = pv

            num_revolutions = count_revolution(initial_state, initial_force, num_steps)

            revolution[i][j] = num_revolutions
    
    revolution = np.array(revolution)
    df = pd.DataFrame(revolution, index = cart_velocity, columns=pole_velocity)
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.heatmap(df, annot=False, fmt=".1f", cmap="coolwarm")

    # Set axis labels
    ax.set_xlabel("initial pole velocity", fontsize=14)
    ax.set_ylabel("initial cart velocity", fontsize=14)
    ax.set_title("Revolutions", fontsize=16)

    # Set ticks at regular intervals (e.g., every 5 steps)
    step = 5  # Show every 5th tick label

    xticks = np.arange(0, num_steps, step)
    yticks = np.arange(0, num_steps, step)

    ax.set_xticks(xticks + 0.5)
    ax.set_xticklabels([f"{pole_velocity[i]:.2f}" for i in xticks], rotation=45, fontsize=12)

    ax.set_yticks(yticks + 0.5)
    ax.set_yticklabels([f"{cart_velocity[i]:.2f}" for i in yticks], rotation=0, fontsize=12)


    # Show the plot
    plt.show()


generate_heatmap_zoomed(num_steps=50)  

---
## Task 1.2

the delta time can be reduced to make the relationship a lot more linear. Infact, infinitesimal delta time will be perfectly linear. However, this will result in the control being very slow to respons. Also, if everything is linear, there will not be any need for non-linear control. 

In [ ]:

def sweep(num_steps, initial_force):
    """
    Perform a sweep of the CartPole environment for a given number of steps.
    
    Args:
        num_steps (int): The number of steps to simulate.
        initial_force (float): The initial force applied to the cart.
    
    Returns:
        None: plots relation between current state and state after performing 
        action
    """
    state_sweeps = {
        'cart_location': np.linspace(-10, 10, num_steps),
        'cart_velocity': np.linspace(-10, 10, num_steps),
        'pole_angle': np.linspace(-np.pi, np.pi, num_steps),
        'pole_velocity': np.linspace(-15, 15, num_steps)
    }

    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']

    env = CartPole(visual=False)

    # this makes angle a lot more linear
    # env.setSimParams(sim_steps=5, delta_time=0.02)

    state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10), 
             np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)]
    
    y_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }

    for i in range(len(state)):
        title = titles[i]
        for j in state_sweeps[title]:
            state[i] = j
            env.reset()
            env.setState(state)
            env.performAction(initial_force)
            new_state = env.getState()
            y_data[title].append(new_state[i])
        
        plot_x_y(state_sweeps[title], y_data[title], title, title + ' after action')

        # reset state to random values
        state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10), 
             np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)]

sweep(num_steps=100, initial_force=0)

In [ ]:
def difference_sweep(num_steps, initial_force):
    """
    Perform a sweep of the CartPole environment for a given number of steps.
    
    Args:
        num_steps (int): The number of steps to simulate.
        initial_force (float): The initial force applied to the cart.
    
    Returns:
        None: plots relation between current state and state after performing 
        action
    """
    state_sweeps = {
        'cart_location': np.linspace(-10, 10, num_steps),
        'cart_velocity': np.linspace(-10, 10, num_steps),
        'pole_angle': np.linspace(-np.pi, np.pi, num_steps),
        'pole_velocity': np.linspace(-15, 15, num_steps)
    }

    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']

    env = CartPole(visual=False)

    # this makes angle a lot more linear
    # env.setSimParams(sim_steps=5, delta_time=0.02)

    state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10), 
             np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)]
    
    y_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }

    for i in range(len(state)):
        title = titles[i]
        for j in state_sweeps[title]:
            state[i] = j
            env.reset()
            env.setState(state)
            env.performAction(initial_force)
            new_state = env.getState()
            y_data[title].append(new_state[i] - state[i])
        
        # plot_x_y(state_sweeps[title], y_data[title], title, "change in " + title)

        # reset state to random values
        state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10), 
             np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)]
        
    plot_state(state_sweeps, y_data)


difference_sweep(num_steps=100, initial_force=0)

In [ ]:
def difference_sweep_2d(num_steps, initial_force):
    """
    Perform a sweep of the CartPole environment for a given number of steps.
    
    Args:
        num_steps (int): The number of steps to simulate.
        initial_force (float): The initial force applied to the cart.
    
    Returns:
        None: plots relation between current state and state after performing 
        action
    """
    state_sweeps = {
        'cart_location': np.random.uniform(-10, 10, num_steps),
        'cart_velocity': np.random.uniform(-10, 10, num_steps),
        'pole_angle': np.random.uniform(-np.pi, np.pi, num_steps),
        'pole_velocity': np.random.uniform(-15, 15, num_steps)
    }

    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']

    env = CartPole(visual=False)

    state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10), 
             np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)]


    for i in range(len(state)):
        title_i = titles[i]
        for j in range(i+1,len(state)):
            title_j = titles[j]
            
            z_data_x = []
            z_data_y = []
            for k in range(len(state_sweeps[title_i])):
                state[i] = state_sweeps[title_i][k]
                state[j] = state_sweeps[title_j][k]
                env.reset()
                env.setState(state)
                env.performAction(initial_force)
                new_state = env.getState()
                z_data_x.append(new_state[i] - state[i])
                z_data_y.append(new_state[j] - state[j])

            plt.figure()
            tcf1 = plt.tricontourf(state_sweeps[title_i], state_sweeps[title_j], z_data_x, levels = 10)
            plt.title(f'Contours of change in {title_i}\n')
            plt.xlabel(title_i)
            plt.ylabel(title_j)
            plt.colorbar(tcf1)
            plt.show()

            plt.figure()
            tcf2 = plt.tricontourf(state_sweeps[title_i], state_sweeps[title_j], z_data_y, levels = 10)
            plt.title(f'Contours of change in {title_j}\n')
            plt.xlabel(title_i)
            plt.ylabel(title_j)
            plt.colorbar(tcf2)
            plt.show()

            # reset state to random values
            state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10), 
                np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)]

difference_sweep_2d(num_steps=10000, initial_force=0)

---
## Task 1.3

In [4]:
def generate_data_rollout(num_steps, initial_force):
    """ 
    Generate data from Cartpole environment for training a model.
    Args:
        num_steps (int): The number of steps to simulate.
        initial_force (float): The initial force applied to the cart.
    Returns:
        X (np.ndarray): The input data, a 2D array of shape (num_steps, 4).
        Y (np.ndarray): The output data, a 2D array of shape (num_steps, 4).
    """
    # initial_state = (np.random.uniform(-10, 10), np.random.uniform(-10, 10), np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15))
    initial_state = [0, 0, np.pi, 5]
    # [[0, 0, np.pi, 5], [0, 0, np.pi, 14], [0, 1, np.pi, 8], [0, 10, np.pi, 14], [0, -2, 3, 0]]
    data = rollout(initial_state=initial_state, initial_force=initial_force, num_steps=num_steps, visual=False)

    X = convert_dict_to_array(data)
    Y = np.array([X[i+1] - X[i] for i in range(X.shape[0] - 1)])

    # remove the last element of X
    X = X[:-1]
    
    # print("shape of X:", X.shape, "\nshape of Y:", Y.shape)
    return X, Y

def generate_data_random(num_steps, initial_force):
    env = CartPole(visual=False)
    env.reset()
    x_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }

    y_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }
    for i in range(num_steps):
        initial_state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10),
                         np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)]
        env.reset()
        env.setState(initial_state)
        env.performAction(initial_force)

        # remap the angle to be between -pi and pi
        # env.remap_angle()
        
        next_state = env.getState()
    
        x_data['cart_location'].append(initial_state[0])
        x_data['cart_velocity'].append(initial_state[1])
        x_data['pole_angle'].append(initial_state[2])
        x_data['pole_velocity'].append(initial_state[3])

        y_data['cart_location'].append(next_state[0] - initial_state[0])
        y_data['cart_velocity'].append(next_state[1] - initial_state[1])
        y_data['pole_angle'].append(next_state[2] - initial_state[2])
        y_data['pole_velocity'].append(next_state[3] - initial_state[3])

    X = convert_dict_to_array(x_data)
    Y = convert_dict_to_array(y_data)
    
    # print("shape of X:", X.shape, "\nshape of Y:", Y.shape)
    return X, Y

def generate_data_random2(num_steps, initial_force = 0):
    np.random.seed(4)
    
    # set the random seed and create the sobol sequence generator
    sobol_engine = scipy.stats.qmc.Sobol( d=4, seed=4 )

    # get 512 initial states spaced in the recommended ranges
    X = (sobol_engine.random_base2( m=int(np.log2(num_steps)) ) - 0.5 ) * 2 * np.array( [ 10, 20, np.pi, 20 ] )

    env = CartPole(visual=False)
    env.reset()
    Y = []

    for i, initial_state in enumerate(X):
        env.reset()
        env.setState(initial_state)
        env.performAction(initial_force)
        # env.remap_angle()
        next_state = env.getState()

        Y.append(next_state - initial_state)

    Y = np.array(Y)  

    return X, Y

In [ ]:
X, Y = generate_data_random(num_steps = 500, initial_force = 0)

In [ ]:
# calculate average condition number

sum_k = 0

for i in range(20):   
    X, Y = generate_data_random(num_steps = 500, initial_force = 0)
    condition_number = np.linalg.cond((X.T @ X))
    sum_k += condition_number

average_condition_number = sum_k/20
print(average_condition_number)

In [ ]:
def linear_regression(X, Y, intercept=False):
    """
    Perform linear regression on the given data.
    
    Args:
        X (numpy.ndarray): The input data.
        Y (numpy.ndarray): The output data.
    
    Returns:
        None: plots the linear regression results.
    """
    # Add a column of ones to X for the intercept term
    if intercept:
        X = np.hstack((np.ones((X.shape[0], 1)), X))
    
    # Calculate the weights using the normal equation
    # W = np.linalg.inv(X.T @ X) @ X.T @ Y
    W = np.linalg.lstsq(X, Y)[0]
    
    # Make predictions
    Y_pred = X @ W

    return W, Y_pred

W, Y_pred = linear_regression(X, Y, intercept=False)

In [ ]:
def difference_sweep_predicted(num_steps, initial_force, W):
    
    state_sweeps = {
        'cart_location': np.linspace(-10, 10, num_steps),
        'cart_velocity': np.linspace(-10, 10, num_steps),
        'pole_angle': np.linspace(-np.pi, np.pi, num_steps),
        'pole_velocity': np.linspace(-15, 15, num_steps)
    }

    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']

    env = CartPole(visual=False)

    state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10), 
             np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)]
    
    x_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }

    y_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }

    y_data_pred = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }

    actual_next_state_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }

    pred_next_state_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }

    for i in range(len(state)):
        title = titles[i]
        for j in state_sweeps[title]:
            state[i] = j
            x_data[title].append(np.array(state))

            env.reset()
            env.setState(state)
            env.performAction(initial_force)
            env.remap_angle()
            
            new_state = env.getState()
            y_data[title].append(np.array(new_state) - np.array(state))
            actual_next_state_data[title].append(np.array(new_state))

            pred_step = np.array(state) @ W
            y_data_pred[title].append(pred_step)
            pred_next_state = np.array(state) + pred_step
            pred_next_state[2] = remap_angle(pred_next_state[2])
            pred_next_state_data[title].append(pred_next_state)

        # reset state to random values
        state = np.array([np.random.uniform(-10, 10), np.random.uniform(-10, 10), 
             np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)])
    
    for title in titles:
        # combine state_sweeps as columns of X
        x_data[title] = np.array(x_data[title])
        
        # combine y_data as columns of Y
        y_data[title] = np.array(y_data[title])
        
        # combine y_data_pred as columns of Y_pred
        y_data_pred[title] = np.array(y_data_pred[title])

        # combine actual_next_state_data as columns of Y
        actual_next_state_data[title] = np.array(actual_next_state_data[title])

        # combine pred_next_state_data as columns of Y_pred
        pred_next_state_data[title] = np.array(pred_next_state_data[title])
        
    return x_data, y_data, y_data_pred, actual_next_state_data, pred_next_state_data

x_sweep, y_sweep, y_sweep_pred, actual_next_state_sweep, pred_next_state_sweep = difference_sweep_predicted(num_steps=100, initial_force=0, W=W)

In [ ]:
def plot_linear_regression_sweep_1(x_data, y_data, y_data_pred, actual_next_state_sweep, pred_next_state_sweep, step):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 5))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            
            X = x_data[title]

            if not step:
                Y = actual_next_state_sweep[title]
                Y_pred = pred_next_state_sweep[title]
            
            else:
                Y = y_data[title]
                Y_pred = y_data_pred[title]
                
            ax[i, j].plot(X[:,i * 2 + j], Y[:,i * 2 + j] ,'r-', label = "actual")
            ax[i, j].plot(X[:,i * 2 + j], Y_pred[:,i * 2 + j] ,'b--', label = "pred")
            ax[i, j].set_title(title)
            ax[i, j].set_xlabel("current state")

            if not step:
                ax[i, j].set_ylabel("next state")
            else:
                ax[i, j].set_ylabel("next step")
                ax[i, j].set_xlim(min(X[:,i * 2 + j]) - 0.1, max(X[:,i * 2 + j]) + 0.1)
                ax[i, j].set_ylim(min([-1, Y_pred[:,i * 2 + j].min() - 0.1, Y[:,i * 2 + j].min() - 0.1]), max([1, Y_pred[:,i * 2 + j].max() + 0.1, Y[:,i * 2 + j].max() + 0.1, Y[:, i * 2 + j].max() + 0.1]))

            ax[i, j].grid()
            ax[i, j].legend()
    # center the title on top of the figure
    fig.suptitle("Forecast sweep")
    fig.tight_layout(rect=[0, 0.03, 1, 1])  # Adjust the rect to make space for the title
    plt.show()

def plot_linear_regression_sweep_2(y_data, y_data_pred, actual_next_state_data, pred_next_state_data, step):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 5))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            
            if not step:
                Y = actual_next_state_data[title]
                Y_pred = pred_next_state_data[title]

            else:
                Y = y_data[title]
                Y_pred = y_data_pred[title]

                
            ax[i, j].plot(Y[:,i * 2 + j], Y_pred[:,i * 2 + j] ,'r-', label = "predicted")
            ax[i, j].plot(Y[:,i * 2 + j], Y[:,i * 2 + j] ,'b--', label = "Y = X")
            ax[i, j].set_title(title)

            if not step:
                ax[i, j].set_xlabel("actual state")
                ax[i, j].set_ylabel("pred state")
            else:
                ax[i, j].set_xlabel("actual step")
                ax[i, j].set_ylabel("pred step")
                ax[i, j].set_xlim(min(X[:,i * 2 + j]) - 0.1, max(X[:,i * 2 + j]) + 0.1)
                ax[i, j].set_ylim(min(-1, Y_pred[:,i * 2 + j].min() - 0.1), max(1, Y_pred[:,i * 2 + j].max() + 0.1))
            ax[i, j].legend()
            ax[i, j].grid()
    # center the title on top of the figure
    fig.suptitle("Forecast sweep")
    fig.tight_layout(rect=[0, 0.03, 1, 1])  # Adjust the rect to make space for the title
    plt.show()

plot_linear_regression_sweep_1(x_sweep, y_sweep, y_sweep_pred, actual_next_state_sweep, pred_next_state_sweep, step = True)
plot_linear_regression_sweep_2(y_sweep, y_sweep_pred, actual_next_state_sweep, pred_next_state_sweep, step = False)

---
## Task 1.4

In [ ]:
def forecast(initial_state, num_steps, W):
    """
    Forecast the future state of the CartPole environment using the learned model.

    Args:
        initial_state (list): The initial state of the environment.
        num_steps (int): The number of steps to forecast.
        W (numpy.ndarray): The learned model weights.

    Returns:
        X_forecast (dict): A dictionary containing the forecasted states.
    """

    # initialise the forecasted state
    X_forecast = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }

    state = np.array(initial_state)

    # perform the action for the specified number of steps
    for i in range(num_steps):
        X_forecast['cart_location'].append(state[0])
        X_forecast['cart_velocity'].append(state[1])
        X_forecast['pole_angle'].append(state[2])
        X_forecast['pole_velocity'].append(state[3])

        # forecast the next state
        state = state + (state @ W)

        # remap the angle to be between -pi and pi
        state[2] = remap_angle(state[2])
        
    return X_forecast   

In [ ]:
print(W)

In [ ]:
def plot_data_vs_forecast_time(data, forecast, graph_title):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 5))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            ax[i, j].plot(np.arange(0, len(data[title])), data[title], 'r-', label='actual')
            ax[i, j].plot(np.arange(0, len(forecast[title])), forecast[title], 'b-', label='forecast')
            ax[i, j].set_title(title)
            ax[i, j].set_xlabel('Iterations')
            ax[i, j].set_ylabel(title)
            ax[i, j].grid()
            ax[i, j].legend()
    # center the title on top of the figure
    fig.suptitle(graph_title)
    fig.tight_layout(rect=[0, 0.03, 1, 1])  # Adjust the rect to make space for the title
    plt.show()

# Example initial states for testing
initial_states = [[0, 0, np.pi, 5], [0, 0, np.pi, 14], [0, 1, np.pi, 8], [0, 10, np.pi, 14], [0, -2, 3, 0]]

# set the initial force to 0
initial_force = 0

# set the number of steps to forecast
num_steps = 100

# Forecast the future state of the CartPole environment using the learned model.
for initial_state in initial_states:
    # obtain the forecasted state
    X_forecast = forecast(initial_state, num_steps, W)

    # obtain the actual state
    X_actual = rollout(initial_state, initial_force, num_steps, visual=False)

    # plot with graph title as initial state formatted to 2 decimal places
    graph_title = f"Initial state: {initial_state[0]:.2f}, {initial_state[1]:.2f}, {initial_state[2]:.2f}, {initial_state[3]:.2f}"
    plot_data_vs_forecast_time(X_actual, X_forecast, graph_title=graph_title)

---
## Task 2.1

Large sigma_j --> the basis function is not sensitive to that dimension as the 
sum of squares contributes nothing to basis function

lambda is a regulariser --> helps reduce overfitting
need trial and error to get the correct value of lambda

These are actually gaussian processes (number of basis functions goes to infinity)

more trial and error to choose sigmas for basis functions. rule of thumb width: standard deviation of data * scalar, where scalar can also be tuned

import jax.numpy as jnp

jit will compile code so the second time the code is run, the compiled code is run instead of redoing compilation (use @jit decorator)

Jax allows for GPU using device (check if there is mac)

Be ready for JAX to gove errors

using both sine and coside allows you to reconstruct the angle and should give really good fit

use as much data as possible, training will take a couple of minutes so use JIT

do not use for loops with JAX. use JAX scan to replace for loops (not needed for week 2, useful for week 3)

In [5]:
def get_variance(X):
    # Calculate the covariance matrix
    return np.var(X, axis=0)

def kernel(X, X_prime, sigma):
    """
    Compute the Gaussian kernel between two sets of data points.
    Args:
        X (numpy.ndarray): The first set of data points. (N, D)
        X_prime (numpy.ndarray): The second set of data points. (M, D)
        sigma (float): The bandwidth parameter for the Gaussian kernel. (D,)
    """
    X_e = np.expand_dims(X, axis=1)  # (N, 1, D)
    X_prime_e = np.expand_dims(X_prime, axis=0)  # (1, M, D)
    diff = X_e - X_prime_e  # (N, M, D)

    # remap angle to be between -pi and pi
    diff[..., 2] = np.sin(diff[..., 2]/2)
    scaled_squared_diff = (diff ** 2)/(2 * sigma ** 2) # (N, M, D)

    K = np.exp(-np.sum(scaled_squared_diff, axis=-1))  # (N, M)
    return K


def tikhonov_solve(K, regularisation_matrix, Y,lamb):
    """
    Solve the Tikhonov regularization problem.
    
    Args:
        K (numpy.ndarray): The kernel matrix. (N, M)
        regularisation_matrix (numpy.ndarray): The regularization matrix. (M, M)
        Y (numpy.ndarray): The output data. (N, D)
        lamb (float): The regularization parameter.
    
    Returns:
        numpy.ndarray: The weights of the model.
    """
    Y_solve = (K.T) @ Y  # (N, D)

    regularisation_term = lamb * regularisation_matrix # (M, M)

    X_solve = ((K.T) @ K) + regularisation_term # (M, M)

    alpha = np.linalg.lstsq(X_solve, Y_solve, rcond=None)[0]  # (M, D)
    
    return alpha

def train_nonlinear_models(X, Y, M, lamb, sigma, kernel_fn):
  

    # choose M random points from X
    # indices = np.random.choice(X.shape[0], M, replace=False)
    # X_prime = X[indices]
    X_prime = X[:M] # (M, D)

    # Create the kernel matrix
    K = kernel_fn(X, X_prime, sigma) # (N, M)

    # Create the regularization matrix
    regularisation_matrix = kernel_fn(X_prime, X_prime, sigma) # (M, M)

    # solve the Tikhonov regularization problem
    alpha = tikhonov_solve(K, regularisation_matrix, Y, lamb)

    return alpha, X_prime, K

def plot_actual_pred_iterations(X_actual, X_forecast, graph_title):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 6))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            ax[i, j].plot(np.arange(0, len(X_actual[:, i * 2 + j])), X_actual[:, i * 2 + j], 'r-', label='actual')
            ax[i, j].plot(np.arange(0, len(X_forecast[:, i * 2 + j])), X_forecast[:, i * 2 + j], 'b--', label='forecast')
            # ax[i, j].set_title(title)
            ax[i, j].set_xlabel('Iterations')
            ax[i, j].set_ylabel(title)
            ax[i, j].grid()
            ax[i, j].legend()
    
    # center the title on top of the figure
    fig.suptitle(graph_title)
    fig.tight_layout(rect=[0, 0.03, 1, 1])  # Adjust the rect to make space for the title
    plt.rcParams.update({'font.size': 16})
    plt.show()

def plot_actual_pred_time(X_actual, X_forecast, graph_title):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 6))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            ax[i, j].plot(0.1 * np.arange(0, len(X_actual[:, i * 2 + j])), X_actual[:, i * 2 + j], 'r-', label='actual')
            ax[i, j].plot(0.1 * np.arange(0, len(X_forecast[:, i * 2 + j])), X_forecast[:, i * 2 + j], 'b--', label='forecast')
            # ax[i, j].set_title(title)
            ax[i, j].set_xlabel('time')
            ax[i, j].set_ylabel(title)
            ax[i, j].grid()
            ax[i, j].legend()
    
    # center the title on top of the figure
    fig.suptitle(graph_title)
    fig.tight_layout(rect=[0, 0.03, 1, 1])  # Adjust the rect to make space for the title
    plt.rcParams.update({'font.size': 16})
    plt.show()

def plot_fit(Y_actual, Y_pred, graph_title):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 8))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            # make scatter plot dots smaller
            # colour the scatter plot dots dark blue
            ax[i, j].scatter(Y_actual[:, i * 2 + j], Y_pred[:, i * 2 + j], label='pred', s=2, color='darkblue')
            ax[i, j].plot(Y_actual[:, i * 2 + j], Y_actual[:, i * 2 + j], 'r--', label='Y = X')
            # ax[i, j].set_title(title)
            ax[i, j].set_xlabel('actual change in state')
            ax[i, j].set_ylabel(title)
            ax[i, j].grid()
            ax[i, j].legend()

    # center the title on top of the figure
    fig.suptitle(graph_title)
    fig.tight_layout(rect=[0, 0.03, 1, 1])  # Adjust the rect to make space for the title
    plt.rcParams.update({'font.size': 15})
    plt.show()
    plt.close()

def forecast_nonlinear(initial_state, initial_force, num_steps, alpha, sigma, X_prime, kernel_fn):
    
    # obtain the actual state
    X_actual = convert_dict_to_array(rollout(initial_state, initial_force, num_steps, visual=False))

    current_state = np.array(initial_state)
    X_forecast = [current_state.copy()]
    for i in range(num_steps):
        # calculate the kernel for the current state
        K = kernel_fn(np.expand_dims(current_state, axis=0), X_prime, sigma)

        Y_pred = K @ alpha
        current_state = (current_state + Y_pred).flatten()

        # remap the angle to be between -pi and pi PURELY FOR PLOTTING
        # The original state is still used for the forecast
        remapped_state = current_state.copy()
        remapped_state[2] = remap_angle(remapped_state[2])
        X_forecast.append(remapped_state)

    X_forecast = np.array(X_forecast)
    
    plot_actual_pred_iterations(X_actual, X_forecast, graph_title=f"Forecast for initial state: {initial_state[0]:.2f}, {initial_state[1]:.2f}, {initial_state[2]:.2f}, {initial_state[3]:.2f}")
    plot_actual_pred_time(X_actual, X_forecast, graph_title=f"Forecast for initial state: {initial_state[0]:.2f}, {initial_state[1]:.2f}, {initial_state[2]:.2f}, {initial_state[3]:.2f}")


# ----------------------------------------------------------------------------
# NOT USED
def plot_fit2(X_actual, Y_actual, Y_pred, graph_title):
    plt.figure(figsize=(10, 6))
    plt.scatter(X_actual[:, 0], Y_actual[:, 0],label='actual')
    plt.scatter(X_actual[:, 0], Y_pred[:, 0], label='pred')
    plt.show()

def plot_fit3(Y_actual, Y_pred, graph_title):
    plt.figure(figsize=(10, 6))
    plt.scatter(Y_actual[:, -1], Y_pred[:, -1], label='pred')
    plt.plot(Y_actual[:, -1], Y_actual[:, -1], 'b--', label='y = x')
    plt.title(graph_title)
    plt.xlabel("actual change in state")
    plt.ylabel("pred change in state")
    plt.show()

def plot_actual_pred2(X_actual, X_forecast, graph_title):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 5))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            ax[i, j].plot(X_actual[:, i * 2 + j], X_forecast[:, i * 2 + j], 'r-', label='forecast')
            ax[i, j].plot(X_actual[:, i * 2 + j], X_actual[:, i * 2 + j], 'b--', label='Y = X')
            ax[i, j].set_title(title)
            ax[i, j].set_xlabel('actual state')
            ax[i, j].set_ylabel(title)
            ax[i, j].grid()
            ax[i, j].legend()
    
    # center the title on top of the figure
    fig.suptitle(graph_title)
    fig.tight_layout(rect=[0, 0.03, 1, 1])  # Adjust the rect to make space for the title
    plt.show()
    plt.close()


In [29]:
N, M = 4096, 1024
lamb = 1e-4
# lamb = 1.61541790e-03
# generate training data
X, Y = generate_data_random(num_steps=N, initial_force=0)

# Get the standard deviation of X
sigma = get_std(X)
# sigma = np.array([8.08589689e+00, 8.20378912e+00, 1.00000000e+00, 7.01860938e+00]) # from jax
# sigma = jnp.array([7.205328,   7.8390459, 1.0,         7.25186536]) # from jax

# train model
alpha, X_prime, K = train_nonlinear_models(X, Y, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel)

# predict using training set
Y_pred = K @ alpha

# this plot does not give much information
# plot_fit(X, Y, Y_pred, graph_title="Fit of the model")

plot_fit(Y, Y_pred, graph_title="Change in state")

# Example initial states for testing
initial_states = [[0, -2, np.pi, 4],[0, 0, np.pi, 5], [0, 0, np.pi, 14], [0, 1, np.pi, 8], [0, 10, np.pi, 14], [0, -2, 3, 0]]
# initial_states = [[0, -2, 3, 0]]

for initial_state in initial_states:
    forecast_nonlinear(initial_state, initial_force=0, num_steps=100, alpha=alpha, sigma=sigma, X_prime=X_prime, kernel_fn=kernel)

In [30]:
def plot_M(N, X, Y, lamb, sigma, kernel_fn):
    """
    Train nonlinear models with varying M and plot the results.
    """
    mse = []
    Ms = [10 * 2**i for i in range(0, int(np.log2(N/10))+1)] + [N]  # M values to test
    for M in Ms:
        alpha, X_prime, K = train_nonlinear_models(X, Y, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_fn)
        Y_pred = K @ alpha
        mse.append(np.mean((Y - Y_pred) ** 2))
    
    plt.figure(figsize=(10, 6))
    # increase font size of all text in the plot
    plt.rcParams.update({'font.size': 16})
    plt.plot(Ms, mse, label='MSE')
    # mark the points with dots
    plt.scatter(Ms, mse)
    # label the final point
    
    plt.title("Mean Squared Error vs M")
    plt.xlabel("M")
    plt.ylabel("Mean Squared Error")
    plt.xscale("log")
    plt.legend()
    plt.grid()
    plt.show()

    print("Best M:", Ms[np.argmin(mse)], "with MSE:", np.min(mse))
    

plot_M(N, X, Y, lamb, sigma, kernel_fn=kernel)


Best M: 2560 with MSE: 0.01290983376233296


---
## Task 2.2

In [105]:
jax.config.update("jax_enable_x64", True)

def kernel_j(X, X_prime, sigma):
    """
    Compute the Gaussian kernel between two sets of data points.
    Args:
        X (numpy.ndarray): The first set of data points. (N, D)
        X_prime (numpy.ndarray): The second set of data points. (M, D)
        sigma (float): The bandwidth parameter for the Gaussian kernel. (D,)
    """
    X_e = jnp.expand_dims(X, axis=1)  # (N, 1, D)
    X_prime_e = jnp.expand_dims(X_prime, axis=0)  # (1, M, D)
    diff = X_e - X_prime_e  # (N, M, D)

    # remapped diff has the same first 2 dmensions and the last dimension as diff, the 3rd dimension is diff_angle
    diff = diff.at[..., 2].set(jnp.sin(diff[..., 2]/2))
    scaled_squared_diff = (diff ** 2)/(2 * sigma ** 2) # (N, M, D)

    K = jnp.exp(-jnp.sum(scaled_squared_diff, axis=-1))  # (N, M)
    return K

def tikhonov_solve_j(K, regularisation_matrix, Y,lamb):
    """
    Solve the Tikhonov regularization problem.
    
    Args:
        K (numpy.ndarray): The kernel matrix. (N, M)
        regularisation_matrix (numpy.ndarray): The regularization matrix. (M, M)
        Y (numpy.ndarray): The output data. (N, D)
        lamb (float): The regularization parameter.
    
    Returns:
        numpy.ndarray: The weights of the model.
    """
    Y_solve = (K.T) @ Y  # (N, D)

    regularisation_term = lamb * regularisation_matrix # (M, M)
    X_solve = ((K.T) @ K) + regularisation_term # (M, M)

    alpha = jnp.linalg.lstsq(X_solve, Y_solve, rcond=None, numpy_resid=True)[0]  # (M, D)
    return alpha

def train_nonlinear_models_j(X, Y, M, lamb, sigma, kernel_fn):
    # choose M points from X
    X_prime = X[:M] # (M, D)

    # Create the kernel matrix
    K = kernel_fn(X, X_prime, sigma) # (N, M)

    # Create the regularization matrix
    regularisation_matrix = kernel_fn(X_prime, X_prime, sigma) # (M, M)

    alpha = tikhonov_solve_j(K, regularisation_matrix, Y, lamb)
    return alpha, X_prime, K


In [32]:
N_train, N_test, M = 4096, 2048, 1024

X, Y = generate_data_random(num_steps=N_train+N_test, initial_force=0)
X = jnp.array(X)
Y = jnp.array(Y)
X_prime = X[:M]

X_train = X[:N_train]
Y_train = Y[:N_train]

X_val = X[-N_test:]
Y_val = Y[-N_test:]

def loss(parameters):
    lamb = parameters[0]
    sigma = parameters[1:]
    # train model
    alpha, X_prime, _= train_nonlinear_models_j(X_train, Y_train, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_j)

    # predict using validation set
    K_val = kernel_j(X_val, X_prime, sigma)
    Y_pred = K_val @ alpha

    mse = jnp.mean((Y_val - Y_pred) ** 2)
    return mse

# create a function that calculates the gradient of the loss function using jax.grad
grad_loss = jax.grad(loss)

initial_lamb = 1E-4
# initial_sigma = get_std(X)
initial_sigma = jnp.array([6, 6, 2, 6])
initial_hyperparameters = jnp.array([initial_lamb, initial_sigma[0], initial_sigma[1], initial_sigma[2], initial_sigma[3]])

losses = [loss(initial_hyperparameters)]

def callback(intermediate_result):
    print(intermediate_result)
    losses.append(intermediate_result.fun)


bounds = [(1E-6, 1E-1)] + [(1, 10)] * 4  # bounds for lamb and sigma
res = scipy.optimize.minimize(loss, x0=initial_hyperparameters, method='L-BFGS-B', jac=grad_loss, bounds=bounds, callback=callback)

 fun: 0.02156570244895218
   x: [ 9.470e-05  6.000e+00  6.000e+00  1.999e+00  6.000e+00]
 fun: 0.021072178569997893
   x: [ 1.000e-06  6.018e+00  6.018e+00  1.951e+00  6.012e+00]
 fun: 0.008083023958953838
   x: [ 1.000e-06  6.307e+00  6.319e+00  1.154e+00  6.215e+00]
 fun: 0.00626494437423588
   x: [ 1.616e-04  6.480e+00  6.499e+00  1.000e+00  6.335e+00]
 fun: 0.005930178427511657
   x: [ 3.033e-04  6.571e+00  6.592e+00  1.000e+00  6.398e+00]
 fun: 0.005508184047323983
   x: [ 6.304e-04  6.802e+00  6.833e+00  1.000e+00  6.560e+00]
 fun: 0.005250136608618767
   x: [ 1.050e-03  7.123e+00  7.166e+00  1.000e+00  6.784e+00]
 fun: 0.005245188902081439
   x: [ 1.352e-03  7.391e+00  7.444e+00  1.000e+00  6.971e+00]
 fun: 0.00521584911873402
   x: [ 2.270e-03  8.466e+00  8.561e+00  1.000e+00  7.721e+00]
 fun: 0.004814830429634167
   x: [ 1.605e-03  9.486e+00  9.620e+00  1.000e+00  8.432e+00]
 fun: 0.004814830429634167
   x: [ 1.605e-03  9.486e+00  9.620e+00  1.000e+00  8.432e+00]


In [33]:
print("optimal lambda:", res.x[0])
print("optimal sigma:", res.x[1:])
print("initial loss", losses[0])
print("Final loss:", res.fun)

def plot_loss(losses):
    plt.figure(figsize=(10, 6))
    plt.rcParams.update({'font.size': 16})
    plt.plot(losses, label='Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Loss over iterations')
    plt.grid()
    plt.legend()
    plt.show()

plot_loss(losses)


optimal lambda: 0.0016048473099244775
optimal sigma: [9.48587713 9.61961829 1.         8.43234787]
initial loss 0.021592418006843736
Final loss: 0.004814830429634167


In [34]:
N, M = 4096, 1024
lamb = res.x[0]

# generate training data
X, Y = generate_data_random(num_steps=N, initial_force=0)

# Get the standard deviation of X
sigma = res.x[1:]

# train model
alpha, X_prime, K = train_nonlinear_models(X, Y, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel)

# predict using training set
Y_pred = K @ alpha

plot_fit(Y, Y_pred, graph_title="Change in state")

# Example initial states for testing
initial_states = [[0, -2, np.pi, 4],[0, 0, np.pi, 5], [0, 0, np.pi, 14], [0, 1, np.pi, 8], [0, 10, np.pi, 14], [0, -2, 3, 0]]

for initial_state in initial_states:
    forecast_nonlinear(initial_state, initial_force=0, num_steps=100, alpha=alpha, sigma=sigma, X_prime=X_prime, kernel_fn=kernel)

---
## Task 2.3

### Use sin and cos

In [9]:
def kernel_expanded(X, X_prime, sigma):
    # create new X where 2 additional dimensions are added, replacing the angle with sin and cos
    # angle dimension is removed
    X_new = np.hstack((X[:,0:2], np.sin(X[:, 2:3]), np.cos(X[:, 2:3]), X[:, 3:]))  # (N, D+1)
    X_prime_new = np.hstack((X_prime[:,0:2], np.sin(X_prime[:, 2:3]), np.cos(X_prime[:, 2:3]), X_prime[:, 3:]))  # (M, D+1)

    X_e = np.expand_dims(X_new, axis=1)  # (N, 1, D+1)
    X_prime_e = np.expand_dims(X_prime_new, axis=0)  # (1, M, D+1)

    diff = X_e - X_prime_e  # (N, M, D+1)
    scaled_squared_diff = (diff ** 2)/(2 * sigma ** 2) # (N, M, D+1)

    K = np.exp(-np.sum(scaled_squared_diff, axis=-1))  # (N, M)
    return K
    

In [10]:
# before optimisation
N, M, lamb = 4096, 1024, 1E-4

# generate training data
X, Y = generate_data_random(num_steps=N, initial_force=0)

# Get the standard deviation of X
x_sigma = get_std(X)
std_sine, std_cos = (0.125)**0.5, (0.125)**0.5  # standard deviation for sine and cosine
sigma = np.array([x_sigma[0], x_sigma[1], std_sine, std_cos, x_sigma[-1]])

# train model
alpha, X_prime, K = train_nonlinear_models(X, Y, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_expanded)

# predict using training set
Y_pred = K @ alpha

# plot_fit(X, Y, Y_pred, graph_title="Fit of the model")
plot_fit(Y, Y_pred, graph_title="Change in state")

# Example initial states for testing
initial_states = [[0, -2, np.pi, 4], [0, 0, np.pi, 5], [0, 0, np.pi, 14], [0, 1, np.pi, 8], [0, 10, np.pi, 14], [0, 0, 0.1, 0]]

for initial_state in initial_states:
    forecast_nonlinear(initial_state, initial_force=0, num_steps=100, alpha=alpha, sigma=sigma, X_prime=X_prime, kernel_fn=kernel_expanded)

### Perform optimisation

In [91]:
@jax.jit
def kernel_expanded_j(X, X_prime, sigma):
    # create new X where 2 additional dimensions are added, replacing the angle with sin and cos
    # angle dimension is removed
    X_new = jnp.hstack((X[:,0:2], jnp.sin(X[:, 2:3]), jnp.cos(X[:, 2:3]), X[:, 3:]))  # (N, D+1)
    X_prime_new = jnp.hstack((X_prime[:,0:2], jnp.sin(X_prime[:, 2:3]), jnp.cos(X_prime[:, 2:3]), X_prime[:, 3:]))  # (M, D+1)

    X_e = jnp.expand_dims(X_new, axis=1)  # (N, 1, D+1)
    X_prime_e = jnp.expand_dims(X_prime_new, axis=0)  # (1, M, D+1)

    diff = X_e - X_prime_e  # (N, M, D+1)
    scaled_squared_diff = (diff ** 2)/(2 * sigma ** 2) # (N, M, D+1)

    K = jnp.exp(-jnp.sum(scaled_squared_diff, axis=-1))  # (N, M)
    return K

In [14]:
N_train, N_test, M = 4096, 2048, 1024

X, Y = generate_data_random(num_steps=N_train+N_test, initial_force=0)
X = jnp.array(X)
Y = jnp.array(Y)
X_prime = X[:M]

X_train = X[:N_train]
Y_train = Y[:N_train]

X_val = X[-N_test:]
Y_val = Y[-N_test:]

def loss(parameters):
    lamb = parameters[0]
    sigma = parameters[1:]
    # train model
    alpha, X_prime, _= train_nonlinear_models_j(X_train, Y_train, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_expanded_j)

    # predict using validation set
    K_val = kernel_expanded_j(X_val, X_prime, sigma)
    Y_pred = K_val @ alpha

    mse = jnp.mean((Y_val - Y_pred) ** 2)
    return mse

# create a function that calculates the gradient of the loss function using jax.grad
grad_loss = jax.grad(loss)

initial_lamb = 1E-4

x_sigma = get_std(X)
# initial_sigma = jnp.array([6, 6, 0.5, 0.5, 6])
initial_sigma = jnp.array([x_sigma[0], x_sigma[1], std_sine, std_cos, x_sigma[-1]])
initial_hyperparameters = jnp.array([initial_lamb, initial_sigma[0], initial_sigma[1], initial_sigma[2], initial_sigma[3], initial_sigma[4]])

losses = [loss(initial_hyperparameters)]

def callback(intermediate_result):
    print(intermediate_result)
    losses.append(intermediate_result.fun)


bounds = [(1E-6, 1E-1)] + [(0, 10)] * 5  # bounds for lamb and sigma
res = scipy.optimize.minimize(loss, x0=initial_hyperparameters, method='L-BFGS-B', jac=grad_loss, bounds=bounds, callback=callback)

 fun: 0.005405604240802274
   x: [ 2.374e-02  5.796e+00  5.813e+00  3.693e-01  3.718e-01  8.622e+00]
 fun: 0.0049689512828856456
   x: [ 2.179e-02  5.798e+00  5.814e+00  3.842e-01  3.883e-01  8.622e+00]
 fun: 0.0034281633411655304
   x: [ 3.601e-03  5.812e+00  5.830e+00  4.718e-01  4.637e-01  8.622e+00]
 fun: 0.0028591045272973745
   x: [ 1.000e-06  5.825e+00  5.845e+00  5.475e-01  5.250e-01  8.622e+00]
 fun: 0.0027421845614860533
   x: [ 1.666e-02  5.850e+00  5.874e+00  7.002e-01  6.487e-01  8.621e+00]
 fun: 0.0024960521880978814
   x: [ 6.855e-03  5.849e+00  5.873e+00  6.890e-01  6.355e-01  8.621e+00]
 fun: 0.0024277652259825314
   x: [ 4.339e-03  5.851e+00  5.875e+00  6.989e-01  6.391e-01  8.621e+00]
 fun: 0.0023337154254061887
   x: [ 2.183e-03  5.860e+00  5.886e+00  7.454e-01  6.624e-01  8.621e+00]
 fun: 0.002252053726113791
   x: [ 2.589e-04  5.893e+00  5.923e+00  9.111e-01  7.469e-01  8.619e+00]
 fun: 0.0022302581044300485
   x: [ 1.303e-03  5.902e+00  5.933e+00  9.549e-01  7.69

In [ ]:
print("optimal lambda:", res.x[0])
print("optimal sigma:", res.x[1:])
print("initial loss", losses[0])
print("Final loss:", res.fun)
print("number of iterations:", len(losses) - 1)

def plot_loss(losses):
    plt.figure(figsize=(10, 6))
    plt.rcParams.update({'font.size': 16})
    plt.plot(losses, label='Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Loss over iterations')
    plt.grid()
    plt.legend()
    plt.show()

plot_loss(losses)

optimal lambda: 0.0008223560132280528
optimal sigma: [6.14331378 6.2057124  1.88010443 0.70520437 8.60437336]
initial loss 0.0057695140578041685
Final loss: 0.0018479414329824722
number of iterations: 22


In [16]:
# optimised hyperparameters
N, M, lamb = 4096, 1024, res.x[0]

# generate training data
X, Y = generate_data_random(num_steps=N, initial_force=0)

# Get the standard deviation of X
sigma = res.x[1:]

# train model
alpha, X_prime, K = train_nonlinear_models(X, Y, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_expanded)

# predict using training set
Y_pred = K @ alpha

# plot_fit(X, Y, Y_pred, graph_title="Fit of the model")
plot_fit(Y, Y_pred, graph_title="Change in state")

# Example initial states for testing
initial_states = [[0, -2, np.pi, 4], [0, 0, np.pi, 5], [0, 0, np.pi, 14], [0, 1, np.pi, 8], [0, 10, np.pi, 14], [0, 0, 0.1, 0]]
# initial_states = [[0, -2, 3, 0]]
for initial_state in initial_states:
    forecast_nonlinear(initial_state, initial_force=0, num_steps=100, alpha=alpha, sigma=sigma, X_prime=X_prime, kernel_fn=kernel_expanded)

In [17]:
non_linear_model_sin = {
    'lambda': lamb,
    'sigma': sigma,
    'alpha': alpha,
    'X_prime': X_prime,
}

# Save the model
with open('non_linear_model_sin.pkl', 'wb') as f:
    pickle.dump(non_linear_model_sin, f)

### linear model

In [ ]:
def expand_state(X):
    """
    Expand the state by replacing the angle with sine and cosine.
    Args:
        X (numpy.ndarray): The input state. (N, D)
    Returns:
        numpy.ndarray: The expanded state. (N, D+1)
    """
    return np.hstack((X[:,0:2], np.sin(X[:, 2:3]), np.cos(X[:, 2:3]), X[:, 3:]))  # (N, D+1)



def generate_data_random_sin(num_steps, initial_force):
    env = CartPole(visual=False)
    env.reset()
    
    x_data, y_data = [], []

    for i in range(num_steps):
        initial_state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10), np.sin(np.random.uniform(-np.pi, np.pi)), np.cos(np.random.uniform(-np.pi, np.pi)), np.random.uniform(-15, 15)]
        env.reset()
        env.setState(initial_state)
        env.performAction(initial_force)
        
        next_state = env.getState()

        x_data.append([initial_state[0], initial_state[1], np.sin(initial_state[2]), np.cos(initial_state[2]), initial_state[3]])
        y_data.append([next_state[0] - initial_state[0], next_state[1] - initial_state[1], next_state[2] - initial_state[2], next_state[3] - initial_state[3]])

    X = np.array(x_data)
    Y = np.array(y_data)
    
    print("shape of X:", X.shape, "\nshape of Y:", Y.shape)
    return X, Y


def forecast_linear_sin(initial_state, initial_force, num_steps, W):
    
    # obtain the actual state
    X_actual = convert_dict_to_array(rollout(initial_state, initial_force, num_steps, visual=False))
    
    # current_state = expand_state(np.array([np.array(initial_state)]))[0]
    current_state = np.array(initial_state)
    current_state_e = [initial_state[0], initial_state[1], np.sin(initial_state[2]), np.cos(initial_state[2]), initial_state[3]]
    X_forecast = [np.array(initial_state)]
    for i in range(num_steps):
        # calculate the kernel for the current state
        current_state_e = np.array([current_state[0], current_state[1], np.sin(current_state[2]), np.cos(current_state[2]), current_state[3]])
        Y_pred = current_state_e @ W

        current_state = (current_state + Y_pred).flatten()
        # remap the angle to be between -pi and pi PURELY FOR PLOTTING
        # The original state is still used for the forecast
        remapped_state = current_state.copy()
        remapped_state[2] = remap_angle(remapped_state[2])
        X_forecast.append(remapped_state)

    X_forecast = np.array(X_forecast)
    
    plot_actual_pred_iterations(X_actual, X_forecast, graph_title=f"Forecast for initial state: {initial_state[0]:.2f}, {initial_state[1]:.2f}, {initial_state[2]:.2f}, {initial_state[3]:.2f}")
    return X_actual, X_forecast


X, Y = generate_data_random_sin(num_steps=4096, initial_force=0)
W, Y_pred = linear_regression(X, Y, intercept=False)
print("shape of W:", W.shape)

# initial_states = [[0, -2, np.pi, 4], [0, 0, np.pi, 5], [0, 0, np.pi, 14], [0, 1, np.pi, 8], [0, 10, np.pi, 14], [0, -2, 3, 0]]
initial_states = [[0, -2, 3, 0]]
for initial_state in initial_states:
    forecast_linear_sin(initial_state, initial_force=0, num_steps=100, W=W)

---
# Task 3.1


In [ ]:
def generate_data_random_force(num_steps, max_force=20):
    env = CartPole(visual=False, max_force=max_force)
    env.reset()
    x_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': [],
        'force': []
    }

    y_data = {
        'cart_location': [],
        'cart_velocity': [],
        'pole_angle': [],
        'pole_velocity': []
    }
    for i in range(num_steps):
        initial_state = [np.random.uniform(-10, 10), np.random.uniform(-10, 10),
                         np.random.uniform(-np.pi, np.pi), np.random.uniform(-15, 15)]
        initial_force = np.random.uniform(-env.max_force, env.max_force)
        env.reset()
        env.setState(initial_state)
        env.performAction(initial_force)

        # remap the angle to be between -pi and pi
        # env.remap_angle()
        
        next_state = env.getState()
    
        x_data['cart_location'].append(initial_state[0])
        x_data['cart_velocity'].append(initial_state[1])
        x_data['pole_angle'].append(initial_state[2])
        x_data['pole_velocity'].append(initial_state[3])
        x_data['force'].append(initial_force)

        y_data['cart_location'].append(next_state[0] - initial_state[0])
        y_data['cart_velocity'].append(next_state[1] - initial_state[1])
        y_data['pole_angle'].append(next_state[2] - initial_state[2])
        y_data['pole_velocity'].append(next_state[3] - initial_state[3])

    X = convert_dict_to_array(x_data)
    Y = convert_dict_to_array(y_data)
    
    # print("shape of X:", X.shape, "\nshape of Y:", Y.shape)
    return X, Y

def forecast_nonlinear_force(initial_state, num_steps, alpha, sigma, X_prime, kernel_fn, max_force=20):
    x_state = initial_state[:-1]  # exclude the force from the state
    initial_force = initial_state[-1]  # force is the last element of the state
    # obtain the actual state
    X_actual = convert_dict_to_array(rollout(x_state, initial_force, num_steps, visual=False, max_force=max_force))

    current_state = np.array(initial_state)
    current_x_state = np.array(x_state)
    X_forecast = [current_x_state.copy()]
    for i in range(num_steps):
        # calculate the kernel for the current state
        K = kernel_fn(np.expand_dims(current_state, axis=0), X_prime, sigma)

        Y_pred = K @ alpha
        current_x_state = (current_state[:-1] + Y_pred).flatten()
        current_state = np.concatenate([current_x_state, current_state[-1:]]) # keep the force unchanged

        # remap the angle to be between -pi and pi PURELY FOR PLOTTING
        # The original state is still used for the forecast
        remapped_state = current_x_state.copy()
        remapped_state[2] = remap_angle(remapped_state[2])
        X_forecast.append(remapped_state)

    X_forecast = np.array(X_forecast)
    
    plot_actual_pred_iterations(X_actual, X_forecast, graph_title=f"Forecast for initial state: {initial_state[0]:.2f}, {initial_state[1]:.2f}, {initial_state[2]:.2f}, {initial_state[3]:.2f} with force {initial_state[4]:.2f}")
    plot_actual_pred_time(X_actual, X_forecast, graph_title=f"Forecast for initial state: {initial_state[0]:.2f}, {initial_state[1]:.2f}, {initial_state[2]:.2f}, {initial_state[3]:.2f} with force {initial_state[4]:.2f}")

In [117]:
N_train, N_test, M = 4096, 2048, 1024

X, Y = generate_data_random_force(num_steps=N_train+N_test)
X = jnp.array(X)
Y = jnp.array(Y)
X_prime = X[:M]

X_train = X[:N_train]
Y_train = Y[:N_train]

X_val = X[-N_test:]
Y_val = Y[-N_test:]

def loss(parameters):
    lamb = parameters[0]
    sigma = parameters[1:]
    # train model
    alpha, X_prime, _= train_nonlinear_models_j(X_train, Y_train, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_expanded_j)

    # predict using validation set
    K_val = kernel_expanded_j(X_val, X_prime, sigma)
    Y_pred = K_val @ alpha

    mse = jnp.mean((Y_val - Y_pred) ** 2)
    return mse

# create a function that calculates the gradient of the loss function using jax.grad
grad_loss = jax.grad(loss)

initial_lamb = 1E-4
max_force = 10
std_force = max_force / (3**0.5)  # standard deviation for force
x_sigma = get_std(X)
# initial_sigma = jnp.array([6, 6, 0.5, 0.5, 6])
std_sine, std_cos = (0.125)**0.5, (0.125)**0.5  # standard deviation for sine and cosine
initial_sigma = jnp.array([x_sigma[0], x_sigma[1], std_sine, std_cos, x_sigma[-1], std_force])
initial_hyperparameters = jnp.array([initial_lamb] + initial_sigma.tolist())

losses = [loss(initial_hyperparameters)]

def callback(intermediate_result):
    print(intermediate_result)
    losses.append(intermediate_result.fun)



bounds = [(1E-6, 1E-1)] + [(0, 30)] + [(0, 40)] + [(0, 1)] * 2 +[(0, 10)] + [(0, max_force*3)]  # bounds for lamb and sigma
res = scipy.optimize.minimize(loss, x0=initial_hyperparameters, method='L-BFGS-B', jac=grad_loss, bounds=bounds, callback=callback)

 fun: 0.12149847913986678
   x: [ 1.000e-06  5.810e+00  5.865e+00  5.689e-01  5.726e-01  9.994e+00
        5.802e+00]
 fun: 0.11200358594098687
   x: [ 5.017e-03  5.850e+00  5.902e+00  6.737e-01  6.288e-01  9.979e+00
        5.837e+00]
 fun: 0.10216939679031292
   x: [ 1.000e-06  5.965e+00  6.016e+00  8.180e-01  7.067e-01  9.939e+00
        5.947e+00]
 fun: 0.09675290610335338
   x: [ 1.000e-01  6.195e+00  6.246e+00  1.000e+00  7.572e-01  9.861e+00
        6.169e+00]
 fun: 0.08443038873926925
   x: [ 6.247e-02  6.403e+00  6.454e+00  1.000e+00  7.654e-01  9.791e+00
        6.371e+00]
 fun: 0.04831389491956825
   x: [ 1.000e-06  7.397e+00  7.445e+00  1.000e+00  6.420e-01  9.459e+00
        7.326e+00]
 fun: 0.027686133935499978
   x: [ 4.469e-02  8.834e+00  8.876e+00  1.000e+00  4.751e-01  8.969e+00
        8.710e+00]
 fun: 0.015209702795340594
   x: [ 1.000e-06  9.993e+00  1.003e+01  9.378e-01  4.835e-01  8.569e+00
        9.829e+00]
 fun: 0.010599022574134639
   x: [ 1.000e-06  1.097e+0

In [114]:
print("optimal lambda:", res.x[0])
print("optimal sigma:", res.x[1:])
print("initial loss", losses[0])
print("Final loss:", res.fun)
print("number of iterations:", len(losses) - 1)

def plot_loss(losses):
    plt.figure(figsize=(10, 6))
    plt.rcParams.update({'font.size': 16})
    plt.plot(losses, label='Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Loss over iterations')
    plt.grid()
    plt.legend()
    plt.show()

plot_loss(losses)

optimal lambda: 0.0002626221654183028
optimal sigma: [28.16510448 30.          0.77844056  0.49904862  5.85409741 26.06744468]
initial loss 0.18865416682746916
Final loss: 0.0006249635892481723
number of iterations: 59


In [116]:
print(res)

  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: 0.0006249635892481723
        x: [ 2.626e-04  2.817e+01  3.000e+01  7.784e-01  4.990e-01
             5.854e+00  2.607e+01]
      nit: 59
      jac: [-9.105e-06 -1.698e-05 -1.472e-05 -2.490e-07  2.030e-07
             1.203e-06 -3.719e-08]
     nfev: 68
     njev: 68
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>


In [ ]:
# N, M, lamb = 4096, 1024, res.x[0]
N, M, lamb = 4096, 1024, 4.543e-03 # mine
# N, M, lamb = 4096, 1024, 4.918e-04 # andrew

# generate training data
X, Y = generate_data_random_force(num_steps=N)

# Get the standard deviation of X
sigma = res.x[1:]
# sigma = np.array([1.000e+01,  1.000e+01,  9.916e-01,  6.063e-01, 7.005e+00,  2.000e+01]) # mine
# sigma = np.array([15.41,  1.413e+01,  5.24,  0.97, 7.356,  13.52])    # andrew

# train model
alpha, X_prime, K = train_nonlinear_models(X, Y, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_expanded)

# predict using training set
Y_pred = K @ alpha

# plot_fit(X, Y, Y_pred, graph_title="Fit of the model")
plot_fit(Y, Y_pred, graph_title="Change in state")


# Example initial states for testing
initial_states = [[0, -2, np.pi, 4, 1], [0, 0, np.pi, 5, 2], [0, 0, np.pi, 0, 10], [0, 0, 0.1, 0, 8]]
# initial_states = [[0, 0, np.pi, 0, 15]]
for initial_state in initial_states:
    forecast_nonlinear_force(initial_state, num_steps=100, alpha=alpha, sigma=sigma, X_prime=X_prime, kernel_fn=kernel_expanded)

non_linear_model_sin_force = {
    'lambda': res.x[0],
    'sigma': res.x[1:],
    'alpha': alpha,
    'X_prime': X_prime,
}



In [120]:
non_linear_model_sin_force = {
    'lambda': res.x[0],
    'sigma': res.x[1:],
    'alpha': alpha,
    'X_prime': X_prime,
}


---
## Task 3.2


### Objective

In order to optimize a policy, we need to define, mathematically, what we want to achieve. That is captured in an _objective function_ (also called _loss function_), a measure of how close we are to the desired behaviour. In the present case, we want the pole to be upright, so we could use the _loss function_

$$
l(X) = -\cos\theta
$$

But it is better, and more general, to define a _target state_, $X_0$, which we want the system to achieve, and use a loss function that increases when the distance of the state from the target is larger. The following loss function achieves this, 

$$
l(X) = 1- e^{-|X-X_0|^2/2\sigma_l^2}
$$
where $\sigma_l$ is a scaling factor (you could introduce a separate one for each component of the state variable). The target state for the cartpole system is $X_0 = [0,0,0,0]$. The advantage of this form is that for large departures from the target, the loss is independent of the state. This expresses the notion that if the pole is far away from being upright and stationary, we do not much care what it is doing. The above loss functions are for a given state. We wish to keep the pole upright continuously, so the total loss of a trajectory should be a time integral (sum, in practice) of the pointwise loss of the state over some interval,

$$
L = \sum_{i=1}^N l(X_i)
$$

The `CartPole` class contains a function to evaluate the above pointwise loss $l(X)$. 

We start with defining a linear policy, 

$$
p(X) = \bf{p} \cdot X
$$

with unknown coefficent vector ${\bf p}$. 

#### Task 3.2

Write code to evaluate the loss function for the trajectory of a rollout - use a short time horizon, but enough to capture 1-2 oscillation periods. Before any optimisation, _visualise_ the loss function as you vary some parameters in ${\bf p}$, using similar 1D and 2D scans that you did for Task 1, i.e. varying just one or two elements of  ${\bf p}$ and plotting the loss as a function of those elements. 

Given a fixed initial state, optimise the unknowns in the policy to minimise the loss function under the evolution given by the true model dynamics. You can do this by using gradients given by jax and replacing calls to numpy in Cartpole.py with calls to jax.numpy. To speed up your code, you will need to use jax.scan instead of the for loop and jax.git. Use the `scipy.optimize` package for optimization as before. Find elements of ${\bf p}$ that are able to stabilize the pole when started just slightly displaced from the upright unstable position. Then try for the case where the pole is in its downward position. For these experiments, you will need to tune the $\sigma_l$ parameters in the loss to obtain good results. Which values of $\sigma_l$ did you choose? You will also have to change the max_force varible in Cartpole.py. Which value did you choose? Now try to start from the downward stable position, what happens in this case?

Plot the time evolution of the variables under the policy as predicted by the dynamics to demonstrate that the pole is kept upright

In [19]:
def loss_policy(state, target, sigma):
    delta = (state - target)/sigma
    exponent = 0.5 * delta.T @ delta
    return 1 - np.exp(-exponent)

def loss_rollout_linear(initial_state, sigma, num_steps, P, target, max_force):
    env = CartPole(visual=False, max_force=max_force)
    env.reset()    
    # Set the initial state
    env.setState(initial_state)

    loss_cum = loss_policy(initial_state, target, sigma)
    # loss_cumulative = [loss_cum]

    # Perform the action for the specified number of steps
    for step in range(num_steps):
        prev_state = env.getState()

        # Perform the action
        force = P @ prev_state
        env.performAction(force)

        # remap the angle to be between -pi and pi
        env.remap_angle()

        curr_state = env.getState()
        loss_cum += loss_policy(curr_state, target, sigma)
        # loss_cumulative.append(loss_cum)

    return loss_cum

initial_state = np.array([0, 0, 0.1, 0])
sigma = np.array([10, 10, 10, 10])
num_steps = 20
target = np.array([0, 0, 0, 0])
P = np.array([1, 1, 1, 1])  # Example policy matrix
max_force=5.0
loss_cum = loss_rollout_linear(initial_state=initial_state, sigma=sigma, num_steps=num_steps, P=P, target=target, max_force=max_force)

print("Cumulative loss:", loss_cum)

Cumulative loss: 5.8192350567562015


In [140]:
def sweep_loss_policy(initial_state, sigma, num_steps, target, max_force):
    # create a 2 by 2 grid of plots
    fig, ax = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('Cumulative Loss vs Policy Parameters')

    P = np.array([0, 0, 0, 0])  # Example policy matrix
    P_sweep = np.linspace(-10, 10, 50)  # Sweep range for P
    
    loss_list = {i: [] for i in range(len(P))}
    for i in range(len(P)):
        P_temp = P.copy()  # Create a temporary copy of P
        for j in P_sweep:
            P_temp[i] = j
            loss_cum = loss_rollout_linear(initial_state=initial_state, sigma=sigma, num_steps=num_steps, P=P_temp, target=target, max_force=max_force)
            loss_list[i].append(loss_cum)

        ax[i // 2, i % 2].plot(P_sweep, loss_list[i], label='Loss', color='b', linestyle=':', marker='o')
        ax[i // 2, i % 2].set_xlabel(f'Policy Parameter {i}')
        ax[i // 2, i % 2].set_ylabel('Cumulative Loss')
        ax[i // 2, i % 2].grid()

    plt.tight_layout()
    plt.show()

initial_state = np.array([0, 0, 0.1, 0])
sigma = np.array([10, 10, 10, 10])
num_steps = 20
target = np.array([0, 0, 0, 0]) # compare with all 0 case to explain ehat happens when 1 parameter changes
max_force = 5.0
sweep_loss_policy(initial_state, sigma, num_steps, target, max_force)

In [25]:
initial_state = jnp.array([0, 0, 0.1, 0])
sigma = jnp.array([10, 10, 10, 10])
# sigma = jnp.array([5.8, 5.8, 1.5, 8.5])  # Example sigma values
num_steps = 25
target = jnp.array([0, 0, 0, 0])
max_force = 8

@jax.jit
def loss_policy_js(state, target, sigma):
    delta = (state - target) / sigma
    exponent = 0.5 * jnp.dot(delta, delta)
    return 1 - jnp.exp(-exponent)

@jax.jit
def loss_rollout_linear_js(P):
    def scan_step(state, _):
        force = P @ state
        next_state = cartpole_step(state, force, max_force=max_force)
        loss = loss_policy_js(next_state, target, sigma)
        return next_state, loss

    init_loss = loss_policy_js(initial_state, target, sigma)
    _, losses = jax.lax.scan(scan_step, initial_state, None, length=num_steps)
    return init_loss + losses.sum()


initial_p = jnp.array([1, 1, 1, 1])  # initial policy parameters for upright initial starting position

grad_loss_linear_js = jax.grad(loss_rollout_linear_js)

losses = [loss_rollout_linear_js(initial_p)]
print("Initial loss:", losses[0])

def callback(intermediate_result):
    print("Iteration:", len(losses))
    print("P:", intermediate_result.x)
    print("Loss:", intermediate_result.fun)
    print()
    losses.append(intermediate_result.fun)

res = scipy.optimize.minimize(loss_rollout_linear_js, x0=initial_p, method='L-BFGS-B', jac=grad_loss_linear_js, callback=callback, bounds=[(-30, 30)] * len(initial_p))

Initial loss: 11.730081878367615
Iteration: 1
P: [1.0194611  0.90596451 1.08966288 1.42778601]
Loss: 11.438371432326754

Iteration: 2
P: [1.13763751 0.83514644 2.40303865 5.40689033]
Loss: 2.58968033154469

Iteration: 3
P: [1.01962007 0.78446937 2.55969119 5.70627998]
Loss: 2.329312200804371

Iteration: 4
P: [0.93427383 0.76159971 2.598113   5.69037867]
Loss: 2.2823112259164873

Iteration: 5
P: [0.64182394 0.75607878 2.60098455 5.16008555]
Loss: 1.9232091394211521

Iteration: 6
P: [-0.10986049  1.15136738  3.26620521  5.19637243]
Loss: 0.9039452394020098

Iteration: 7
P: [-0.76900608  1.48814999  3.78734291  5.0843024 ]
Loss: 0.5277695281040773

Iteration: 8
P: [-1.19114585  1.75534678  4.19681908  5.07779055]
Loss: 0.3571018730347868

Iteration: 9
P: [-1.70178613  2.09412353  4.71504515  5.07444659]
Loss: 0.23573776744558794

Iteration: 10
P: [-2.159717    2.41572402  5.21606381  5.08346209]
Loss: 0.17401203728027181

Iteration: 11
P: [-2.55348974  2.71443771  5.71331074  5.10731747]


In [26]:
def rollout_linear(initial_state, num_steps, P, max_force, visual=False):
    env = CartPole(visual=visual, max_force=max_force)
    env.reset()    
    # Set the initial state
    env.setState(initial_state)

    X = []

    # Perform the action for the specified number of steps
    for step in range(num_steps):
        prev_state = env.getState()
        X.append(prev_state.copy())

        # Perform the action
        force = P @ prev_state
        env.performAction(force)

        # remap the angle to be between -pi and pi
        env.remap_angle()

    return np.array(X)

def plot_policy(X, target, graph_title):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 6))
    titles = ['cart_location', 'cart_velocity', 'pole_angle', 'pole_velocity']
    for i in range(2):
        for j in range(2):
            title = titles[i * 2 + j]
            ax[i, j].plot(np.arange(0, len(X[:, i * 2 + j])), X[:, i * 2 + j], 'r-', label='policy')
            ax[i, j].plot(np.arange(0, len(X[:, i * 2 + j])), [target[i * 2 + j]] * len(X[:, i * 2 + j]), 'b--', label='target')
            # ax[i, j].set_title(title)
            ax[i, j].set_xlabel('Iterations')
            ax[i, j].set_ylabel(title)
            ax[i, j].grid()
            ax[i, j].legend()
    
    # center the title on top of the figure
    fig.suptitle(graph_title)
    fig.tight_layout(rect=[0, 0.03, 1, 1])  # Adjust the rect to make space for the title
    plt.rcParams.update({'font.size': 16})
    plt.show()

initial_state = np.array([0, 0, 0.1, 0])
# sigma = jnp.array([5.8, 5.8, 1.5, 8.5]) 
num_steps = 100
target = np.array([0, 0, 0, 0])
P = res.x  # optimised policy matrix
max_force = 8
X = rollout_linear(initial_state=initial_state, num_steps=num_steps, P=P, max_force=max_force, visual=True)
plot_policy(X, target, f"Rollout Policy, initial_state: {initial_state}")

---
### Tests for other sigmas

In [ ]:
initial_state = jnp.array([0, 0, 0.1, 0])
num_steps = 20
target = jnp.array([0, 0, 0, 0])
initial_p = jnp.array([1, 1, 1, 1]) 
opt_loss = []
for i in range(1, 12):
    sigma = jnp.array([i, 10, 10, 10])
    print("\n\nsigma:\n", sigma)
    res = scipy.optimize.minimize(loss_rollout_linear_js, x0=initial_p, method='L-BFGS-B', jac=grad_loss_linear_js, bounds=[(-10, 10)] * len(initial_p))
    opt_loss.append(res.fun)




sigma:
 [ 1 10 10 10]
[0.99416163 0.93691398 1.02126157 0.71827885]
11.520321483859595

[ 0.98984783  0.87565762  0.93010552 -0.10280988]
4.811904681869081

[ 0.98989     0.8762864   0.93098537 -0.09466664]
4.346313739258631

[ 0.98987892  0.87592882  0.93108032 -0.09521886]
4.333750103222909

[ 0.98920485  0.84834215  0.94709876 -0.07651922]
3.7095577785525498

[ 0.98916931  0.84660769  0.94812223 -0.07556213]
3.672805859031084

[ 0.98916838  0.84656152  0.94814996 -0.07553256]
3.671796633669464



sigma:
 [ 2 10 10 10]
[1.00577834 0.95593961 1.05882654 0.96730891]
9.450269919810813

[1.13315896 0.14626276 2.07512919 0.11488974]
6.828166332258985

[1.1333704  0.14491218 2.07682268 0.11350477]
6.828060849481559

[ 1.27825965e+00 -9.40885115e-01  3.39223337e+00 -4.32344426e-04]
3.8882422066904

[ 1.26620221e+00 -8.50297292e-01  3.27305233e+00 -1.30414794e-03]
2.421114544527981

[ 1.26649939e+00 -8.52487415e-01  3.27580492e+00 -1.77389619e-03]
2.4175512229508618

[ 1.27756063 -0.931243

In [10]:
sigma = jnp.array([20, 20, 20, 20])
print("\n\nsigma:\n", sigma)
res = scipy.optimize.minimize(loss_rollout_linear_js, x0=initial_p, method='L-BFGS-B', jac=grad_loss_linear_js, callback=callback, bounds=[(-10, 10)] * len(initial_p))
opt_loss.append(res.fun)



sigma:
 [20 20 20 20]
[0.99852926 0.98655573 1.02927642 1.06804639]
1.8626156471087811

[1.00743149 0.99667099 1.11147449 1.38646064]
1.8172269658468634

[1.59511704 3.25292992 2.5676244  6.83432372]
0.6111047491890416

[0.13216122 2.52990428 3.59238241 7.30794995]
0.12511290540603814

[-0.4052314   2.48279224  3.9287092   6.64564926]
0.0672407124083576

[-0.41953065  2.48314496  3.95176268  6.40295685]
0.05472366163865794

[-1.21764551  2.16965618  4.48391616  5.32288792]
0.03367197550344203

[-1.34777563  2.50556052  4.95096956  5.6084411 ]
0.024333155121549876

[-1.72224854  2.79406609  5.66940784  5.54992243]
0.016749449257246196

[-2.26859823  3.23340844  6.75880706  5.50773703]
0.010584200800321075

[-2.82575288  3.70521265  7.9539504   5.49383159]
0.00714307730249919

[-3.26516942  4.12076801  9.3012308   5.46248492]
0.0053693335572967404

[-3.44405711  4.37362253 10.          5.40364313]
0.0048142914430344685

[-3.08212281  4.23871399 10.          5.42077692]
0.00461183209826

In [8]:
print(opt_loss)

[3.671745170413871, 1.985245861288856, 0.046200523030422325, 0.03469186378264422, 0.028430565087880222, 0.024508923697478147, 0.12965358276794103, 0.01990654811075021, 0.08544893104532847]


---
# Initial state from downhang starts here

In [ ]:
initial_state = np.array([0, 0, np.pi, 0])
sigma = np.array([10, 10, 4, 10])
num_steps = 20
target = np.array([0, 0, 0, 0])
P = np.array([7, 7, 7, 7])  # Example policy matrix
max_force=10.0

loss_cum = loss_rollout_linear(initial_state=initial_state, sigma=sigma, num_steps=num_steps, P=P, target=target, max_force=max_force)

print("Cumulative loss:", loss_cum)
sweep_loss_policy(initial_state, sigma, num_steps, target, max_force)

Cumulative loss: 16.15809550378633


In [137]:
initial_state = jnp.array([0, 0, jnp.pi, 0])
sigma = jnp.array([10, 10, 4, 10])
num_steps = 1000
target = jnp.array([0, 0, 0, 0])
max_force = 10

# best found
# sigma = jnp.array([10, 10, 4, 10])
# num_steps = 75
# max_force = 10
# initial_p = jnp.array([5, 5, 5, 5]) 

# 2nd best found
# sigma = jnp.array([10, 10, 4, 10])
# num_steps = 20
# max_force = 8
# initial_p = jnp.array([5, 5, 5, 5]) 

@jax.jit
def loss_policy_js(state, target, sigma):
    delta = (state - target) / sigma
    exponent = 0.5 * jnp.dot(delta, delta)
    return 1 - jnp.exp(-exponent)

@jax.jit
def loss_rollout_linear_js(P):
    def scan_step(state, _):
        force = P @ state
        next_state = cartpole_step(state, force, max_force=max_force)
        loss = loss_policy_js(next_state, target, sigma)
        return next_state, loss

    init_loss = loss_policy_js(initial_state, target, sigma)
    _, losses = jax.lax.scan(scan_step, initial_state, None, length=num_steps)
    return init_loss + losses.sum()


initial_p = jnp.array([5, 5, 5, 5])  # initial policy parameters for upright initial starting position

grad_loss_linear_js = jax.grad(loss_rollout_linear_js)

losses = [loss_rollout_linear_js(initial_p)]
print("Initial loss:", losses[0])

def callback(intermediate_result):
    print("Iteration:", len(losses))
    print("P:", intermediate_result.x)
    print("Loss:", intermediate_result.fun)
    print()
    losses.append(intermediate_result.fun)

res = scipy.optimize.minimize(loss_rollout_linear_js, x0=initial_p, method='L-BFGS-B', jac=grad_loss_linear_js, callback=callback, bounds=[(-20, 20)] * len(initial_p))

Initial loss: 991.4790631705297
Iteration: 1
P: [4.23613134 2.9379491  5.83387337 6.52203774]
Loss: 982.3683667587937

Iteration: 2
P: [2.01720406 1.61325882 6.49870723 6.51885056]
Loss: 951.2638107780207

Iteration: 3
P: [2.02967984 1.62061861 6.49499543 6.51897184]
Loss: 943.8429529753022

Iteration: 4
P: [2.02975155 1.62070198 6.49487523 6.51885154]
Loss: 943.8235218044637

Iteration: 5
P: [2.02573391 1.63354141 6.49961877 6.51353001]
Loss: 943.3403592113552

Iteration: 6
P: [2.0213557  1.64650483 6.50424323 6.50778641]
Loss: 942.9521088519558

Iteration: 7
P: [2.01677717 1.6587693  6.50736439 6.50083447]
Loss: 942.1991663402598

Iteration: 8
P: [2.01632864 1.65996223 6.50752919 6.50001923]
Loss: 942.1566586536441

Iteration: 9
P: [2.01610208 1.66044674 6.50730125 6.49936867]
Loss: 942.0774019713144

Iteration: 10
P: [2.01455571 1.66365273 6.5051157  6.49436632]
Loss: 941.7248750419077

Iteration: 11
P: [2.01192821 1.66723192 6.49489988 6.48055725]
Loss: 940.7623147209987

Iteration

In [ ]:
num_iter = 100
P_opt = res.x  # optimised policy matrix
# P_opt = [-1.00289127  0.01541063 10.          5.72545855]
initial_state = jnp.array([0, 0, jnp.pi, 0])
X = rollout_linear(initial_state=initial_state, num_steps=num_iter, P=P_opt, max_force=max_force, visual=True)
plot_policy(X, target, f"Rollout Policy, initial_state: {initial_state}")

---
# My own model

In [ ]:
N_train, N_test, M = 4096, 2048, 1024

X, Y = generate_data_random_force(num_steps=N_train+N_test)
X = jnp.array(X)
Y = jnp.array(Y)
X_prime = X[:M]

X_train = X[:N_train]
Y_train = Y[:N_train]

X_val = X[-N_test:]
Y_val = Y[-N_test:]

def loss(parameters):
    lamb = parameters[0]
    sigma = parameters[1:]
    # train model
    alpha, X_prime, _= train_nonlinear_models_j(X_train, Y_train, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_expanded_j)

    # predict using validation set
    K_val = kernel_expanded_j(X_val, X_prime, sigma)
    Y_pred = K_val @ alpha

    mse = jnp.mean((Y_val - Y_pred) ** 2)
    return mse

# create a function that calculates the gradient of the loss function using jax.grad
grad_loss = jax.grad(loss)

initial_lamb = 1E-4
max_force = 10
std_force = max_force / (3**0.5)  # standard deviation for force
x_sigma = get_std(X)
# initial_sigma = jnp.array([6, 6, 0.5, 0.5, 6])
std_sine, std_cos = (0.125)**0.5, (0.125)**0.5  # standard deviation for sine and cosine
initial_sigma = jnp.array([x_sigma[0], x_sigma[1], std_sine, std_cos, x_sigma[-1], std_force])
initial_hyperparameters = jnp.array([initial_lamb] + initial_sigma.tolist())

losses = [loss(initial_hyperparameters)]

def callback(intermediate_result):
    print(intermediate_result)
    losses.append(intermediate_result.fun)



bounds = [(1E-6, 1E-1)] + [(0, 30)] + [(0, 40)] + [(0, 1)] * 2 +[(0, 10)] + [(0, max_force*3)]  # bounds for lamb and sigma
res = scipy.optimize.minimize(loss, x0=initial_hyperparameters, method='L-BFGS-B', jac=grad_loss, bounds=bounds, callback=callback)

In [ ]:
# N, M, lamb = 4096, 1024, res.x[0]
N, M, lamb = 4096, 1024, 4.543e-03 # mine
# N, M, lamb = 4096, 1024, 4.918e-04 # andrew

# generate training data
X, Y = generate_data_random_force(num_steps=N)

# Get the standard deviation of X
sigma = res.x[1:]
# sigma = np.array([1.000e+01,  1.000e+01,  9.916e-01,  6.063e-01, 7.005e+00,  2.000e+01]) # mine
# sigma = np.array([15.41,  1.413e+01,  5.24,  0.97, 7.356,  13.52])    # andrew

# train model
alpha, X_prime, K = train_nonlinear_models(X, Y, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_expanded)

# predict using training set
Y_pred = K @ alpha

# plot_fit(X, Y, Y_pred, graph_title="Fit of the model")
plot_fit(Y, Y_pred, graph_title="Change in state")


# Example initial states for testing
initial_states = [[0, -2, np.pi, 4, 1], [0, 0, np.pi, 5, 2], [0, 0, np.pi, 0, 10], [0, 0, 0.1, 0, 8]]
# initial_states = [[0, 0, np.pi, 0, 15]]
for initial_state in initial_states:
    forecast_nonlinear_force(initial_state, num_steps=100, alpha=alpha, sigma=sigma, X_prime=X_prime, kernel_fn=kernel_expanded)

non_linear_model_sin_force = {
    'lambda': res.x[0],
    'sigma': res.x[1:],
    'alpha': alpha,
    'X_prime': X_prime,
}

In [173]:
initial_state = jnp.array([0, 0, 0.1, 0])
sigma = jnp.array([10, 10, 10, 10])
# sigma = jnp.array([5.8, 5.8, 1.5, 8.5])  # Example sigma values
num_steps = 25
target = jnp.array([0, 0, 0, 0])
max_force = 5

model_x_prime = jnp.array(non_linear_model_sin_force['X_prime'])
model_sigma = jnp.array(non_linear_model_sin_force['sigma'])
model_alpha = jnp.array(non_linear_model_sin_force['alpha'])

@jax.jit
def kernel_expanded_jax(X, X_prime, sigma):
    # create new X where 2 additional dimensions are added, replacing the angle with sin and cos
    # angle dimension is removed
    X_new = jnp.hstack((X[:,0:2], jnp.sin(X[:, 2:3]), jnp.cos(X[:, 2:3]), X[:, 3:]))  # (N, D+1)
    X_prime_new = jnp.hstack((X_prime[:,0:2], jnp.sin(X_prime[:, 2:3]), jnp.cos(X_prime[:, 2:3]), X_prime[:, 3:]))  # (M, D+1)

    X_e = jnp.expand_dims(X_new, axis=1)  # (N, 1, D+1)
    X_prime_e = jnp.expand_dims(X_prime_new, axis=0)  # (1, M, D+1)

    diff = X_e - X_prime_e  # (N, M, D+1)
    scaled_squared_diff = (diff ** 2)/(2 * sigma ** 2) # (N, M, D+1)

    K = jnp.exp(-jnp.sum(scaled_squared_diff, axis=-1))  # (N, M)
    return K

@jax.jit
def loss_policy_jax(state, target, sigma):
    delta = (state - target) / sigma
    exponent = 0.5 * jnp.dot(delta, delta)
    return 1 - jnp.exp(-exponent)

@jax.jit
def loss_rollout_linear_jax(P):
    def scan_step_jax(state, _):
        force = P @ state
        force = max_force * jnp.tanh(force/max_force)

        # add force as the last element to the state
        current_state = jnp.concatenate([state, jnp.array([force])])
        K = kernel_expanded_jax(jnp.expand_dims(current_state, axis=0), model_x_prime, model_sigma)
        Y_pred = K @ model_alpha
        next_state = jnp.ravel(current_state[:-1] + Y_pred)

        loss = loss_policy_jax(next_state, target, sigma)
        return next_state, loss

    init_loss = loss_policy_jax(initial_state, target, sigma)
    _, losses = jax.lax.scan(scan_step_jax, initial_state, None, length=num_steps)
    return init_loss + losses.sum()

initial_p = jnp.array([1, 1, 1, 1])  # initial policy parameters for upright initial starting position

grad_loss_linear_jax = jax.grad(loss_rollout_linear_jax)

losses = [loss_rollout_linear_jax(initial_p)]
print("Initial loss:", losses[0])

def callback(intermediate_result):
    print("Iteration:", len(losses))
    print("P:", intermediate_result.x)
    print("Loss:", intermediate_result.fun)
    print()
    losses.append(intermediate_result.fun)

res = scipy.optimize.minimize(loss_rollout_linear_jax, x0=initial_p, method='L-BFGS-B', jac=grad_loss_linear_jax, callback=callback, bounds=[(-30, 30)] * len(initial_p))

Initial loss: 11.458382241941543
Iteration: 1
P: [0.99982126 1.00064914 1.61637828 2.94520903]
Loss: 8.468525264739952

Iteration: 2
P: [1.07877323 1.80128483 2.58205625 4.9568416 ]
Loss: 3.1553133259331485

Iteration: 3
P: [0.45732689 0.98457717 2.98686655 6.18622001]
Loss: 1.4944850326942603

Iteration: 4
P: [-4.64049532e-03  1.07202593e+00  3.25252633e+00  5.94362921e+00]
Loss: 1.1602948817144243

Iteration: 5
P: [-1.78386987  2.00547932  4.37272646  4.41182545]
Loss: 0.3004923523492823

Iteration: 6
P: [-1.93329554  2.16836438  4.56088901  4.44963118]
Loss: 0.23327490987657984

Iteration: 7
P: [-2.17015716  2.46797327  4.90297277  4.53389308]
Loss: 0.15600826918527722

Iteration: 8
P: [-2.32013512  2.71470527  5.19321712  4.64560194]
Loss: 0.11826184627181247

Iteration: 9
P: [-2.43921268  2.99066502  5.54162504  4.82766003]
Loss: 0.09122473763365135

Iteration: 10
P: [-2.5080372   3.2703984   5.93628627  5.07608635]
Loss: 0.07293233229421292

Iteration: 11
P: [-2.53133109  3.59463

In [232]:
def rollout_linear_force(initial_state, num_steps, P, max_force):
    X_forecast = [initial_state.copy()]
    
    state = initial_state.copy()
    for step in range(num_steps):
        force = P @ state
        force = max_force * np.tanh(force/max_force)
        # add force as the last element to the state
        current_state = jnp.concatenate([state, jnp.array([force])])
        K = kernel_expanded_jax(jnp.expand_dims(current_state, axis=0), model_x_prime, model_sigma)

        Y_pred = K @ model_alpha
        state = jnp.ravel(current_state[:-1] + Y_pred)

        remapped_state = state.copy()
        # remap jax angle to be between -pi and pi PURELY FOR PLOTTING
        remapped_state = np.array([remapped_state[0], remapped_state[1], remap_angle2(remapped_state[2]), remapped_state[3]])
        X_forecast.append(remapped_state)

    return np.array(X_forecast)


initial_state = jnp.array([0, 0, 0.1, 0])
# sigma = jnp.array([5.8, 5.8, 1.5, 8.5]) 
num_steps = 100
target = jnp.array([0, 0, 0, 0])
P = res.x  # optimised policy matrix
max_force = 8
X = rollout_linear_force(initial_state=initial_state, num_steps=num_steps, P=P, max_force=max_force)
plot_policy(X, target, f"Rollout Policy, initial_state: {initial_state}")

---
### upsidedown

In [175]:
N_train, N_test, M = 4096, 2048, 1024

X, Y = generate_data_random_force(num_steps=N_train+N_test)
X = jnp.array(X)
Y = jnp.array(Y)
X_prime = X[:M]

X_train = X[:N_train]
Y_train = Y[:N_train]

X_val = X[-N_test:]
Y_val = Y[-N_test:]

def loss(parameters):
    lamb = parameters[0]
    sigma = parameters[1:]
    # train model
    alpha, X_prime, _= train_nonlinear_models_j(X_train, Y_train, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_expanded_j)

    # predict using validation set
    K_val = kernel_expanded_j(X_val, X_prime, sigma)
    Y_pred = K_val @ alpha

    mse = jnp.mean((Y_val - Y_pred) ** 2)
    return mse

# create a function that calculates the gradient of the loss function using jax.grad
grad_loss = jax.grad(loss)

initial_lamb = 1E-4
max_force = 15
std_force = max_force / (3**0.5)  # standard deviation for force
x_sigma = get_std(X)
# initial_sigma = jnp.array([6, 6, 0.5, 0.5, 6])
std_sine, std_cos = (0.125)**0.5, (0.125)**0.5  # standard deviation for sine and cosine
initial_sigma = jnp.array([x_sigma[0], x_sigma[1], std_sine, std_cos, x_sigma[-1], std_force])
initial_hyperparameters = jnp.array([initial_lamb] + initial_sigma.tolist())

losses = [loss(initial_hyperparameters)]

def callback(intermediate_result):
    print(intermediate_result)
    losses.append(intermediate_result.fun)



bounds = [(1E-6, 1E-1)] + [(0, 30)] + [(0, 40)] + [(0, 1)] * 2 +[(0, 10)] + [(0, max_force*3)]  # bounds for lamb and sigma
res = scipy.optimize.minimize(loss, x0=initial_hyperparameters, method='L-BFGS-B', jac=grad_loss, bounds=bounds, callback=callback)

 fun: 0.07452682084529441
   x: [ 2.078e-02  5.829e+00  5.783e+00  5.471e-01  6.022e-01  9.993e+00
        8.675e+00]
 fun: 0.06940119787146609
   x: [ 5.081e-06  5.846e+00  5.803e+00  6.319e-01  6.290e-01  9.986e+00
        8.689e+00]
 fun: 0.06461554479135118
   x: [ 4.877e-02  5.900e+00  5.866e+00  8.465e-01  6.846e-01  9.962e+00
        8.733e+00]
 fun: 0.061910273871716034
   x: [ 1.759e-02  5.934e+00  5.904e+00  9.138e-01  6.917e-01  9.948e+00
        8.760e+00]
 fun: 0.05970035207135144
   x: [ 1.000e-06  6.000e+00  5.981e+00  1.000e+00  6.963e-01  9.918e+00
        8.812e+00]
 fun: 0.05782939371223952
   x: [ 4.339e-03  6.048e+00  6.036e+00  1.000e+00  6.776e-01  9.897e+00
        8.849e+00]
 fun: 0.04846248196528812
   x: [ 5.984e-03  6.573e+00  6.642e+00  1.000e+00  4.894e-01  9.665e+00
        9.255e+00]
 fun: 0.035910138120118666
   x: [ 1.321e-02  7.071e+00  7.219e+00  1.000e+00  5.431e-01  9.442e+00
        9.646e+00]
 fun: 0.01713724777468697
   x: [ 1.550e-02  8.614e+00

In [176]:
# N, M, lamb = 4096, 1024, res.x[0]
N, M, lamb = 4096, 1024, 4.543e-03 # mine
# N, M, lamb = 4096, 1024, 4.918e-04 # andrew

# generate training data
X, Y = generate_data_random_force(num_steps=N)

# Get the standard deviation of X
sigma = res.x[1:]
# sigma = np.array([1.000e+01,  1.000e+01,  9.916e-01,  6.063e-01, 7.005e+00,  2.000e+01]) # mine
# sigma = np.array([15.41,  1.413e+01,  5.24,  0.97, 7.356,  13.52])    # andrew

# train model
alpha, X_prime, K = train_nonlinear_models(X, Y, M=M, lamb=lamb, sigma=sigma, kernel_fn=kernel_expanded)

# predict using training set
Y_pred = K @ alpha

# plot_fit(X, Y, Y_pred, graph_title="Fit of the model")
plot_fit(Y, Y_pred, graph_title="Change in state")


# Example initial states for testing
initial_states = [[0, -2, np.pi, 4, 1], [0, 0, np.pi, 5, 2], [0, 0, np.pi, 0, 10], [0, 0, 0.1, 0, 8]]
# initial_states = [[0, 0, np.pi, 0, 15]]
for initial_state in initial_states:
    forecast_nonlinear_force(initial_state, num_steps=100, alpha=alpha, sigma=sigma, X_prime=X_prime, kernel_fn=kernel_expanded)

non_linear_model_sin_force_down = {
    'lambda': res.x[0],
    'sigma': res.x[1:],
    'alpha': alpha,
    'X_prime': X_prime,
}

In [298]:
initial_state = jnp.array([0, 0, np.pi, 0])

sigma = jnp.array([10, 8, 4, 10])
# sigma = jnp.array([5.8, 5.8, 1.5, 8.5])  # Example sigma values
num_steps = 50
target = jnp.array([0, 0, 0, 0])
max_force = 10
initial_p = jnp.array([5, 5, 5, 5])  # initial policy parameters for upright initial starting position

'''
Best found

sigma = jnp.array([10, 8, 4, 10])
# sigma = jnp.array([5.8, 5.8, 1.5, 8.5])  # Example sigma values
num_steps = 50
target = jnp.array([0, 0, 0, 0])
max_force = 10
initial_p = jnp.array([5, 5, 5, 5])  # initial policy parameters for upright initial starting position
'''

model_x_prime = jnp.array(non_linear_model_sin_force_down['X_prime'])
model_sigma = jnp.array(non_linear_model_sin_force_down['sigma'])
model_alpha = jnp.array(non_linear_model_sin_force_down['alpha'])

@jax.jit
def kernel_expanded_jax(X, X_prime, sigma):
    # create new X where 2 additional dimensions are added, replacing the angle with sin and cos
    # angle dimension is removed
    X_new = jnp.hstack((X[:,0:2], jnp.sin(X[:, 2:3]), jnp.cos(X[:, 2:3]), X[:, 3:]))  # (N, D+1)
    X_prime_new = jnp.hstack((X_prime[:,0:2], jnp.sin(X_prime[:, 2:3]), jnp.cos(X_prime[:, 2:3]), X_prime[:, 3:]))  # (M, D+1)

    X_e = jnp.expand_dims(X_new, axis=1)  # (N, 1, D+1)
    X_prime_e = jnp.expand_dims(X_prime_new, axis=0)  # (1, M, D+1)

    diff = X_e - X_prime_e  # (N, M, D+1)
    scaled_squared_diff = (diff ** 2)/(2 * sigma ** 2) # (N, M, D+1)

    K = jnp.exp(-jnp.sum(scaled_squared_diff, axis=-1))  # (N, M)
    return K

@jax.jit
def loss_policy_jax(state, target, sigma):
    delta = (state - target) / sigma
    exponent = 0.5 * jnp.dot(delta, delta)
    return 1 - jnp.exp(-exponent)

@jax.jit
def loss_rollout_linear_jax(P):
    def scan_step_jax(state, _):
        force = P @ state
        force = max_force * jnp.tanh(force/max_force)

        # add force as the last element to the state
        current_state = jnp.concatenate([state, jnp.array([force])])
        K = kernel_expanded_jax(jnp.expand_dims(current_state, axis=0), model_x_prime, model_sigma)
        Y_pred = K @ model_alpha
        next_state = jnp.ravel(current_state[:-1] + Y_pred)

        loss = loss_policy_jax(next_state, target, sigma)
        return next_state, loss

    init_loss = loss_policy_jax(initial_state, target, sigma)
    _, losses = jax.lax.scan(scan_step_jax, initial_state, None, length=num_steps)
    return init_loss + losses.sum()

grad_loss_linear_jax = jax.grad(loss_rollout_linear_jax)

losses = [loss_rollout_linear_jax(initial_p)]
print("Initial loss:", losses[0])

def callback(intermediate_result):
    print("Iteration:", len(losses))
    print("P:", intermediate_result.x)
    print("Loss:", intermediate_result.fun)
    print()
    losses.append(intermediate_result.fun)

res = scipy.optimize.minimize(loss_rollout_linear_jax, x0=initial_p, method='L-BFGS-B', jac=grad_loss_linear_jax, callback=callback, bounds=[(-20, 20)] * len(initial_p))

P = res.x  # optimised policy matrix
X = rollout_linear_force(initial_state=initial_state, num_steps=35, P=P, max_force=max_force)
plot_policy(X, target, f"Rollout Policy, initial_state: {initial_state}")

Initial loss: 44.97876788997704
Iteration: 1
P: [4.9997188  4.9991665  4.95835504 5.0001997 ]
Loss: 44.97701146853298

Iteration: 2
P: [ 4.90058011  4.70509924 -9.59255673  5.07009902]
Loss: 43.009347457086285

Iteration: 3
P: [3.40974714 0.16597303 0.41038833 7.3375711 ]
Loss: 33.03232565384911

Iteration: 4
P: [ 2.53143881 -0.59063241  1.14536932  7.81265171]
Loss: 25.974507285665872

Iteration: 5
P: [ 2.53597217 -0.58836054  1.14155556  7.81160962]
Loss: 25.958017149098307

Iteration: 6
P: [ 2.54066786 -0.59363837  1.13866308  7.80955971]
Loss: 25.951252691173444

Iteration: 7
P: [ 2.66571626 -0.774539    1.06179219  7.74965685]
Loss: 25.684747020716987

Iteration: 8
P: [ 3.17936897 -1.47760633  0.78149034  7.50607199]
Loss: 23.0910738242954

Iteration: 9
P: [ 3.18066829 -1.47941265  0.78076403  7.50545652]
Loss: 23.090446652748934

Iteration: 10
P: [ 3.22109292 -1.57388829  0.74839092  7.48427847]
Loss: 22.87799575272503

Iteration: 11
P: [ 3.25104588 -1.66533901  0.71777428  7.467